# 환경 설치

In [1]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np
import re
import pickle
from sklearn.preprocessing import MinMaxScaler

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ! pip install openpyxl==3.0.5

In [2]:
# policy
policy = pd.read_excel('/content/drive/MyDrive/웰로/data/policy_metetable(AIB전달용).xlsx')
policy = policy.where(pd.notnull(policy), None) # Nan to None

# admin
admin = pd.read_csv('/content/drive/MyDrive/웰로/data/wello_admin_답지.csv', error_bad_lines=False)
admin = admin.where(pd.notnull(admin), None) # Nan to None

# user
user = pd.read_excel('/content/drive/MyDrive/웰로/data/유저데이터 샘플리스트 셔플(AIB전달용).xlsx')
user = user.where(pd.notnull(user), None) # Nan to None

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 2439: expected 24 fields, saw 25\n'


In [3]:
policy.head(2)

,정책ID,정책서비스ID,서비스명,소관기관,소관기관유형,소관기관.1,문의처전화번호,소관기관유형.1,온라인신청사이트URL,생애주기,신청절차,선정기준,지원유형,서비스목적,지원내용,지원대상,신청기한_시작,신청기한_끝,상세조회URL,태그-신청절차,태그-자녀,태그-수집유형,태그-가구원,태그-성별,태그-학력,태그-직장,태그-결혼,태그-지원유형,태그-지역(시도),태그-관심상황특성,태그-자녀상세,태그-직장상세,태그-지원유형상세,태그-지역(시군구),태그-대상특성상세,태그-대상특성,태그-대상연령시작,태그-대상연령끝,태그-중위소득시작,태그-중위소득끝
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),교육부,중앙행정기관,교육부,129,중앙정부,http://online.bokjiro.go.kr/,영·유아||아동·청소년||청년||중장년||노년기,신청ㆍ접수(읍면동사무소) - 보장 결정(시군구 사업팀),1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...,현금,누리과정이란 만 3~5세를 대상으로 하는 어린이집 표준보육과정과 유치원 교육과정을 ...,"만 3~5세 누리과정 운영에 필요한 비용 지원 \n - 보육료, 담당교사 처우 개선...",지원대상 : 어린이집을 이용하는 만 3~5세 아동\n- 취학유예 : 만 6세아 초등...,None,None,https://gov.kr/portal/rcvfvrSvc/dtlEx/00000046...,None,None,NaN,None,None,None,None,None,현금,None,None,None,None,None,None,None,None,6,12,0,0
1,2,105100000001,근로·자녀장려금,국세청,중앙행정기관,국세청,None,중앙정부,https://www.hometax.go.kr,None,"○ 개벌 신청 안내받은 경우에는 ARS, 모바일 국세청 홈택스 앱, 인터넷 홈택스 ...","○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 충족하는 자\n\...",현금,○ 근로자녀장려세제\n - 저소득계층의 빈곤 탈출을 지원하고 사회안전망을 확충하기 ...,○ 전년도 연간 부부합산 총 급여액 등(사업소득과 근로소득의 합계)에 따라\n - ...,"○ 전년도에 정해진 범위의 근로소득, 사업소득 또는 종교인소득이 있고 근로장려금과 ...",None,None,https://gov.kr/portal/rcvfvrSvc/dtlEx/10510000...,None,있음,NaN,None,None,None,None,None,현금,None,None,None,None,None,None,None,None,0,99,0,0


In [4]:
user.head(2)

,num,mb_sex,mb_birth,mb_1,mb_2,mb_3,mb_4,mb_5,mb_6,mb_7,mb_8,mb_9,mb_10,mb_11,mb_12,mb_13,num.1,mb_15
0,11726.0,M,19000101.0,충북/청주시,충북/청주시,기타,기타,None,None,None,None,None,None,None,None,None,11726.0,None
1,21745.0,M,19000122.0,경기/고양시,경기/고양시,대학(원) 졸업,근로자,세대주,미혼,없다,no,no,해당없음,해당없음,중위소득 100% 이상,no,21745.0,"취업 지원,근로자 지원,문화생활 지원,주택-부동산 지원"


In [5]:
admin.head(2)

,정책ID,정책서비스ID,서비스명,신청절차,자녀,수집유형,가구원,성별,학력,직장,결혼,지원유형,지역(시도),관심상황특성,자녀상세,직장상세,지원유형상세,지역(시군구),대상특성상세,대상특성,대상연령시작,대상연령끝,중위소득기준-시작,중위소득기준-종료
0,97485,B37003100002,건강관리 상담,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0
1,97484,B37003100001,체육지도자 자격증 발급,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0


In [ ]:
# 정책ID, 정책서비스ID, 서비스명, 소관기관, 소관기관유형, 소관기관.1, 문의처전화번호,
#        소관기관유형.1, 온라인신청사이트URL, 생애주기, 신청절차, 선정기준, 지원유형, 서비스목적,
#        지원내용, 지원대상, 신청기한_시작, 신청기한_끝, 상세조회URL

# 태깅 모델

## user 태깅

### feature에서 unique한 카테고리와 빈도 가져오는 함수

In [112]:
def get_category(df, columns):
  data = df[columns]
  category = {}
  multi = False
  for word in data:
    lst = []
    if word == None:
      word = 'None'
    
    if ',' in word:
      lst = [x.strip() for x in word.split(',')]
    else:
      lst.append(word)
    for i in lst:
      if i in category:
        category[i] += 1
      else:
        category[i] = 1
    if ',' in word:
      multi = True

  category = list(sorted(category.items(), key=lambda item: item[1], reverse=True))
  if multi == True:
    category.append('중복 있음')
  else:
    category.append('중복 없음')
  return category

### mb_sex => 성별

In [ ]:
user.mb_sex.value_counts()

M    12465
F     7545
Name: mb_sex, dtype: int64

In [ ]:
def mb_sex_tagging(text):
  if text == 'M':
    return '남성'
  elif text == 'F':
    return '여성'
  else:
    return '무관'

user['성별'] = user.mb_sex.apply(mb_sex_tagging)
# user.drop(columns = 'mb_sex', inplace=True)
user.성별.value_counts()

남성    12465
여성     7545
Name: 성별, dtype: int64

### mb_birth => 나이

In [ ]:
user.mb_birth.isnull().sum()

0

In [ ]:
user['나이'] = user['mb_birth'].apply(lambda x: 2021 - int(str(x)[:4]))
user.나이

0        121
1        121
2        116
3        102
4        101
        ... 
20005     14
20006     14
20007     14
20008     14
20009     14
Name: 나이, Length: 20010, dtype: int64

### mb_1, mb_2 => 시도, 시군구
거주 지역과 관심 지역을 합치기

In [ ]:
user.head(2)

,num,mb_sex,mb_birth,mb_1,mb_2,mb_3,mb_4,mb_5,mb_6,mb_7,mb_8,mb_9,mb_10,mb_11,mb_12,mb_13,num.1,mb_15,성별,나이
0,11726.0,M,19000101.0,충북/청주시,충북/청주시,기타,기타,None,None,None,None,None,None,None,None,None,11726.0,None,남성,121
1,21745.0,M,19000122.0,경기/고양시,경기/고양시,대학(원) 졸업,근로자,세대주,미혼,없다,no,no,해당없음,해당없음,중위소득 100% 이상,no,21745.0,"취업 지원,근로자 지원,문화생활 지원,주택-부동산 지원",남성,121


In [ ]:
# split  기준으로 나누기
living = user['mb_1']
living1 = []
living2 = []

want = user['mb_2']
want1 = []
want2 = []

for i in range(len(user)):
  living1.append(living[i].split('/')[0])
  living2.append(living[i].split('/')[1])

  want1.append(want[i].split('/')[0])
  want2.append(want[i].split('/')[1])

user['거주_시도'] = living1
user['거주_시군구'] = living2

user['관심_시도'] = want1
user['관심_시군구'] = want2

In [ ]:
user.거주_시도.unique()

array(['충북', '경기', '경남', '대전', '제주', '광주', '서울', '경북', '인천', '충남', '전남',
       '부산', '전북', '대구', '강원', '울산', '세종', ''], dtype=object)

In [ ]:
user.관심_시도.unique()

array(['충북', '경기', '경남', '대전', '제주', '광주', '충남', '경북', '전남', '서울', '인천',
       '부산', '전북', '대구', '강원', '울산', '세종', ''], dtype=object)

In [ ]:
# 시도명 fullname으로 변경
ori = ['충북', '경기', '경남', '대전', '제주', '광주', '서울', '경북', '인천', '충남', '전남',
       '부산', '전북', '대구', '강원', '울산', '세종', '']
change = ['충청북도', '경기도', '경상남도', '대전광역시', '제주특별자치도', '광주광역시', '서울특별시', '경상북도', '인천광역시',
          '충청남도', '전라남도', '부산광역시', '전라북도', '대구광역시', '강원도', '울산광역시', '세종특별자치시', '전국']

for i, j in zip(ori, change):
  user['거주_시도'] = user['거주_시도'].replace(i,j)
  user['관심_시도'] = user['관심_시도'].replace(i,j)


In [ ]:
user.거주_시도.unique()

array(['충청북도', '경기도', '경상남도', '대전광역시', '제주특별자치도', '광주광역시', '서울특별시',
       '경상북도', '인천광역시', '충청남도', '전라남도', '부산광역시', '전라북도', '대구광역시', '강원도',
       '울산광역시', '세종특별자치시', '전국'], dtype=object)

In [ ]:
user.관심_시도.unique()

array(['충청북도', '경기도', '경상남도', '대전광역시', '제주특별자치도', '광주광역시', '충청남도', '경상북도',
       '전라남도', '서울특별시', '인천광역시', '부산광역시', '전라북도', '대구광역시', '강원도', '울산광역시',
       '세종특별자치시', '전국'], dtype=object)

In [ ]:
user.거주_시군구.unique()

array(['청주시', '고양시', '양산시', '여주시', '유성구', '제주시', '서구', '마포구', '평택시',
       '창원시', '경주시', '연천군', '송파구', '중구', '군포시', '밀양시', '금천구', '동작구',
       '고성군', '동대문구', '관악구', '부평구', '아산시', '영주시', '시흥시', '광산구', '수원시',
       '음성군', '용인시', '여수시', '해운대구', '포천군', '광주시', '용산구', '고창군', '의정부시',
       '양평군', '구미시', '달서구', '수영구', '영등포구', '서초구', '계양구', '남양주시', '광진구',
       '목포시', '천안시', '포항시', '전주시', '금정구', '서대문구', '단양군', '남구', '동래구',
       '영암군', '포천시', '칠곡군', '구로구', '상주시', '강서구', '성남시', '연수구', '이천시',
       '수성구', '부산진구', '삼척시', '노원구', '의왕시', '파주시', '강남구', '화성시', '동구',
       '진주시', '거제시', '진해시', '남동구', '화천군', '북구', '서귀포시', '김포시', '김해시',
       '동두천시', '안양시', '논산시', '강동구', '신안군', '성북구', '은평구', '도봉구', '달성구',
       '강북구', '고흥군', '거창군', '세종시 전체', '', '원주시', '오산시', '부천시', '충주시',
       '중랑구', '군산시', '춘천시', '서산시', '안동시', '안산시', '양천구', '영덕군', '양주시',
       '울주군', '성동구', '안성시', '종로구', '해남군', '무안군', '경산시', '연제구', '문경시',
       '진도군', '기장군', '광양시', '인제군', '영월군', '공주시', '속초시', '순천시', '보령시',
       '사하구', '광

In [ ]:
user.관심_시군구.unique()

array(['청주시', '고양시', '양산시', '여주시', '유성구', '제주시', '서구', '아산시', '평택시',
       '창원시', '경주시', '구리시', '순천시', '중구', '군포시', '밀양시', '서초구', '금천구',
       '관악구', '고성군', '동대문구', '부평구', '영주시', '시흥시', '광산구', '수원시', '음성군',
       '용인시', '여수시', '해운대구', '포천군', '광주시', '용산구', '고창군', '의정부시', '양평군',
       '구미시', '달서구', '수영구', '구로구', '영등포구', '광명시', '계양구', '남양주시', '광진구',
       '목포시', '마포구', '천안시', '포항시', '완주군', '금정구', '전주시', '단양군', '남구',
       '송파구', '동래구', '영암군', '안성시', '포천시', '칠곡군', '상주시', '강서구', '성남시',
       '이천시', '수성구', '부산진구', '삼척시', '노원구', '의왕시', '파주시', '강남구', '동구',
       '진주시', '거제시', '남동구', '서대문구', '화천군', '북구', '서귀포시', '김포시', '안산시',
       '김해시', '연수구', '동두천시', '안양시', '강동구', '화성시', '신안군', '울주군', '은평구',
       '도봉구', '달성구', '강북구', '고흥군', '거창군', '세종시 전체', '', '원주시', '오산시',
       '충주시', '중랑구', '성북구', '군산시', '춘천시', '서산시', '안동시', '부천시', '양천구',
       '동작구', '영덕군', '성동구', '양주시', '해남군', '무안군', '경산시', '연제구', '문경시',
       '진도군', '사상구', '기장군', '광양시', '인제군', '영월군', '공주시', '속초시', '보령시',
       '사하구', '담

In [ ]:
area1 = []
area2 = []
for i in range(len(user)):
  x = user['거주_시도'].iloc[i]
  y = user['관심_시도'].iloc[i]
  a = user['거주_시군구'].iloc[i]
  b = user['관심_시군구'].iloc[i]
  
  if x == y:
    area1.append(x)
  else:
    area1.append(x+','+y)
  
  if a == b:
    area2.append(a)
  else:
    area2.append(a+','+b)

In [ ]:
user['시도'] = area1
user['시군구'] = area2

In [ ]:
for i in range(len(user)):
  if user['시군구'].iloc[i] == '':
    user['시군구'].iloc[i] = '전체'

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


### mb_3 => 학력

In [ ]:
user.mb_3.value_counts()

고등학교 졸업     6526
기타          5455
대학(원) 졸업    3606
대학(원) 재학    2932
대학(원) 휴학    1005
Name: mb_3, dtype: int64

In [ ]:
def mb_3_tagging(text):
  if text == '고등학교 졸업':
    return '고등학교졸업'
  elif text == '대학(원) 졸업':
    return '대학(원)졸업'
  elif text == '대학(원) 재학':
    return '대학(원)재학'
  elif text == '대학(원) 휴학':
    return '대학(원)휴학'
  else:
    return '무관'

user['학력'] = user.mb_3.apply(mb_3_tagging)
# user.drop(columns = 'mb_3', inplace=True)
user.학력.value_counts()

고등학교졸업     6526
무관         5941
대학(원)졸업    3606
대학(원)재학    2932
대학(원)휴학    1005
Name: 학력, dtype: int64

In [ ]:
user.학력.isnull().sum()

0

### mb_4 => 직장

- mb_4의 분포도

> 기타: 6796
>
> 근로자: 5287
>
> 대학(원)생: 4103
>
> 구직자: 3779
>
> 프리랜서: 1051
>
> 예비창업자: 491
>
> 기업인/소상공인: 441

In [ ]:
user.mb_4.value_counts()

기타                              6021
근로자                             4133
대학(원)생                          3008
구직자                             2569
프리랜서                             541
                                ... 
대학(원)생,구직자,근로자,예비창업자,프리랜서,기타       1
근로자,예비창업자,기업인/소상공인,기타              1
대학(원)생,구직자,예비창업자,프리랜서,기타           1
구직자,프리랜서,기업인/소상공인                  1
대학(원)생,구직자,근로자,예비창업자,기타            1
Name: mb_4, Length: 98, dtype: int64

In [ ]:
user.mb_4.isnull().sum()

1043

In [ ]:
get_category(user, 'mb_4')

[('기타', 6796),
 ('근로자', 5287),
 ('대학(원)생', 4103),
 ('구직자', 3779),
 ('프리랜서', 1051),
 ('None', 1043),
 ('예비창업자', 491),
 ('기업인/소상공인', 441),
 '중복 있음']

In [ ]:
def mb_4_tagging(text):
  if text == None:
    return '무관'
  else:
    text = text.replace('기타', '무관').replace('근로자', '근로자/직장인').replace('대학(원)생', '대학/대학원생').replace('구직자', '구직자/실업자').replace('프리랜서', '프리랜서').replace('예비창업자', '예비창업자').replace('기업인/소상공인', '소상공인,중소기업')
    return text

user['직장'] = user.mb_4.apply(mb_4_tagging)

get_category(user, '직장')

[('무관', 7839),
 ('근로자/직장인', 5287),
 ('대학/대학원생', 4103),
 ('구직자/실업자', 3779),
 ('프리랜서', 1051),
 ('예비창업자', 491),
 ('소상공인', 441),
 ('중소기업', 441),
 '중복 있음']

### mb_5 => 가구원 유형
- mb_5 분포도

> 세대원:    0.684258
>
> 세대주:    0.240130
>
> None:    0.075612



In [ ]:
user.mb_5.value_counts()

세대원    13692
세대주     4805
Name: mb_5, dtype: int64

In [ ]:
def mb_5_tagging(text):
  if text ==  None:
    return '무관'
  else:
    text = text.replace('세대원', '무주택 세대원,주택 세대원').replace('세대주', '무주택 세대주,주택 세대주')
    return text

user['가구원'] = user.mb_5.apply(mb_5_tagging)
# user.drop(columns = 'mb_5', inplace=True)
user.가구원.value_counts()

무주택 세대원,주택 세대원    13692
무주택 세대주,주택 세대주     4805
무관                 1513
Name: 가구원, dtype: int64

### mb_6 => 결혼
- mb_6 분포도
> 미혼:     0.854523
>
> None:    0.077961
>
> 기혼:     0.067516

In [ ]:
user.mb_6.value_counts()

미혼    17099
기혼     1351
Name: mb_6, dtype: int64

In [ ]:
def mb_6_tagging(text):
  if text == None:
    return '미혼,기혼'
  else:
    text = text.replace('미혼', '미혼').replace('기혼', '기혼')
    return text
  

user['결혼'] = user.mb_6.apply(mb_6_tagging)
# user.drop(columns = 'mb_6', inplace=True)
user.결혼.value_counts()

미혼       17099
미혼,기혼     1560
기혼        1351
Name: 결혼, dtype: int64

### mb_7 => 자녀
- mb_7 분포도
> 없다:     17198
>
> None:     1698
>
> 있다:      1114

In [ ]:
user.mb_7.value_counts()

없다    17198
있다     1114
Name: mb_7, dtype: int64

In [ ]:
def mb_7_tagging(text):
  if text == None:
    return '있음,없음'
  else:
    text = text.replace('없다', '없음').replace('있다', '있음')
    return text

user['자녀'] = user.mb_7.apply(mb_7_tagging)
# user.drop(columns = 'mb_7', inplace=True)
user.자녀.value_counts()

없음       17198
있음,없음     1698
있음        1114
Name: 자녀, dtype: int64

### mb_8 => 자녀 수
- mb_8 분포도
> 1명:       0.484657
>
> 2명:       0.389892
>
> 3명 이상:    0.125451

admin에는 없는 feature 이므로 column 만 변경

In [ ]:
user.rename(columns={'mb_8':'자녀수'}, inplace=True)

### mb_9 => 자녀상세
- mb_9 분포도상세
> no: 17198
>
> 고등학생: 103
 >
 > 성인: 159
 >
 > 영아: 478
 >
 > 유아: 322
 >
 > 중학생: 100
 >
 > 초등학생: 296

In [ ]:
get_category(user, 'mb_9')

[('no', 17198),
 ('None', 1708),
 ('영아', 478),
 ('유아', 322),
 ('초등학생', 296),
 ('성인', 159),
 ('고등학생', 103),
 ('중학생', 100),
 '중복 있음']

In [ ]:
def mb_9_tagging(text):
  if (text == None) or (text=='no'):
    return '해당사항없음'
  else:
    text = text.replace('고등학생', '초중고학생').replace('영유아','잠깐기다려').replace('성인','성인').replace('유아','영유아').replace('영아', '영유아').replace('중학생', '초중고학생').replace('초등학생', '초중고학생').replace('잠깐기다려', '영유아')
    return text

user['자녀상세'] = user.mb_9.apply(mb_9_tagging)
get_category(user, '자녀상세')

[('해당사항없음', 18906), ('영유아', 800), ('초중고학생', 499), ('성인', 159), '중복 있음']

In [ ]:
get_category(user,'자녀상세')

[('해당사항없음', 18906), ('영유아', 800), ('초중고학생', 499), ('성인', 159), '중복 있음']

### mb_10 (특수상황) => 대상특성
> 구호구제대상자: 11
>
> 농축수산인: 100 => A12-02
 >
 > 다문화 가정: 241 => A12-09
 >
 > 다자녀 가정: 1654 => A12-13
 >
 > 독거노인: 12 => A12-14
 >
 > 보훈대상 및 가족: 119 => A12-07
 >
 > 북한이탈주민: 13 => A12-10
 >
 > 소년소녀가장: 125
 >
 > 입양 가정: 28 => A12-15
 >
 > 장애인: 282 => A12-01
 >
 > 저소득층: 2604
 >
 > 한부모/조손 가정: 2242 => A12-11
 >
 > 해당없음: 11691 => A12-00
 >
 > 환자: 135 => A12-03
 >
 >

In [ ]:
get_category(user, 'mb_10')

[('해당없음', 11691),
 ('저소득층', 2604),
 ('None', 2386),
 ('한부모/조손 가정', 2242),
 ('다자녀 가정', 1654),
 ('장애인', 282),
 ('다문화 가정', 241),
 ('환자', 135),
 ('소년소녀가장', 125),
 ('보훈대상 및 가족', 119),
 ('농축수산인', 100),
 ('입양 가정', 28),
 ('북한이탈주민', 13),
 ('독거노인', 12),
 ('구호구제대상자', 11),
 '중복 있음']

In [ ]:
def mb_10_tagging(text):
  if text == None:
    return '해당사항없음'
  elif text == '해당없음':
    return '해당사항없음'
  else:
    text = text.replace('농축수산인',' 농축수산인').replace('다문화 가정', '다문화가족').replace('다자녀 가정','다자녀가정').replace('독거노인', '독거노인').replace('보훈대상 및 가족', '국가유공자').replace('북한이탈주민', '북한이탈주민').replace('입양 가정', '구호구제대상자').replace('장애인', '장애인').replace('한부모/조손 가정', '한부모가정/조손가정').replace('해당없음', '해당사항없음').replace('환자', '질병/부상/질환자')
    return text

user['mb_10+대상특성'] = user.mb_10.apply(mb_10_tagging)
# user.drop(columns = 'mb_10', inplace=True)
get_category(user, 'mb_10+대상특성')

[('해당사항없음', 14077),
 ('저소득층', 2604),
 ('한부모가정/조손가정', 2242),
 ('다자녀가정', 1654),
 ('장애인', 282),
 ('다문화가족', 241),
 ('질병/부상/질환자', 135),
 ('소년소녀가장', 125),
 ('국가유공자', 119),
 (' 농축수산인', 54),
 ('농축수산인', 46),
 ('구호구제대상자', 39),
 ('북한이탈주민', 13),
 ('독거노인', 12),
 '중복 있음']

### mb_11 => 관심상황특성
- 분포도
>결혼예정: 735 => A13-01
>
>구직예정: 3221
>
>은퇴예정: 128 => A13-02
>
>임신예정: 225
>
>입대예정: 1525 => A13-00
>
>제대예정: 517
>
>출생예정: 177
>
>해당없음: 11527

In [ ]:
get_category(user, 'mb_11')

[('해당없음', 11527),
 ('구직 예정', 3221),
 ('None', 2740),
 ('입대 예정', 1525),
 ('결혼 예정', 735),
 ('제대 예정', 517),
 ('임신 예정', 225),
 ('출생 예정', 177),
 ('은퇴 예정', 128),
 '중복 있음']

In [ ]:
def mb_11_tagging(text):
  if text == '해당없음':
    return None
  elif text == None:
    return None
  else:
    text = text.replace('결혼 예정','결혼예정').replace('은퇴 예정', '은퇴예정').replace('입대 예정', '입대예정')
    return text

user['mb_11+관심상황특성'] = user.mb_11.apply(mb_11_tagging)
# user.drop(columns = 'mb_11', inplace=True)
get_category(user, 'mb_11+관심상황특성')

[('None', 14267),
 ('구직 예정', 3221),
 ('입대예정', 1525),
 ('결혼예정', 735),
 ('제대 예정', 517),
 ('임신 예정', 225),
 ('출생 예정', 177),
 ('은퇴예정', 128),
 '중복 있음']

### mb_13 => 상세한 장애 목록인데, admin과 연결 불가. 대상특성상세와 연결 불가. 냅두기

In [ ]:
get_category(user, 'mb_13')

[('no', 17827),
 ('None', 1915),
 ('한부모/조손 가정', 81),
 ('지체', 61),
 ('청각', 34),
 ('뇌병변', 31),
 ('정신', 26),
 ('언어', 19),
 ('해당없음', 16),
 ('시각', 16),
 ('자폐성', 13),
 ('신장', 4),
 ('간', 3),
 ('호흡기', 2),
 ('심장', 2),
 ('간질', 2),
 ('장루', 2),
 '중복 있음']

### mb_14 => 보훈대상 상세인데, admin과 연결 불가. 냅두기

In [ ]:
user.columns

Index(['num', 'mb_sex', 'mb_birth', 'mb_1', 'mb_2', 'mb_3', 'mb_4', 'mb_5',
       'mb_6', 'mb_7', '자녀수', 'mb_9', 'mb_10', 'mb_11', 'mb_12', 'mb_13',
       'num.1', 'mb_15', '성별', '나이', '거주_시도', '거주_시군구', '관심_시도', '관심_시군구',
       '시도', '시군구', '학력', '직장', '가구원', '결혼', '자녀', '자녀상세', 'mb_10+대상특성',
       'mb_11+관심상황특성'],
      dtype='object')

In [ ]:
get_category(user, 'mb_14')

### mb_15: 관심정책. 관심상황특성과 연결 짓기?
>개인금융지원: 6652
>
>교육지원(만8~19세): 4746
>
>근로자 지원: 7297
>
>기업금융지원: 2064
>
>문화생활 지원: 6521
>
>보육지원(만0~7세): 782
>
>성인교육지원: 4557
>
>의료 지원: 4308
>
>주택-부동산 지원: 5141
>
>창업 지원: 3214
>
>취업 지원: 722

In [ ]:
get_category(user, 'mb_15')

[('근로자 지원', 7297),
 ('취업 지원', 7221),
 ('개인금융지원', 6652),
 ('문화생활 지원', 6521),
 ('주택-부동산 지원', 5141),
 ('교육지원(만8~19세)', 4746),
 ('성인교육지원', 4557),
 ('의료 지원', 4308),
 ('None', 3870),
 ('창업 지원', 3214),
 ('기업금융지원', 2064),
 ('보육지원(만0~7세)', 782),
 '중복 있음']

In [ ]:
# 로자 지원
# 취업 지원
# 개인금융지원
# 문화생활 지원
# 주택-부동산 지원
# 교육지원(만8~19세)
# 성인교육지원
# 의료 지원
# 창업 지원
# 기업금융지원
# 보육지원(만0~7세)

### user_final 로 저장

In [ ]:
user.columns

In [ ]:
user = user[['num', '성별', '나이', '시도', '시군구', '학력', '직장', '가구원', '결혼', '자녀', '자녀상세', '자녀수', 'mb_10', 'mb_10+대상특성', 'mb_11', 'mb_11+관심상황특성', 'mb_12', 'mb_13', 'mb_14', 'mb_15']]

In [ ]:
user_final0117 = open('/content/drive/MyDrive/웰로/data/user_final0117.pkl', 'wb')
pickle.dump(user,user_final0117)

## policy 태깅

### 대상연령시작, 대상연령끝
from 생애주기, 지원대상

In [6]:
# policy + admin 중 대상연령에 필요한 feature
policy_admin_age = pd.merge(left=policy, right=admin, how='inner', on='정책ID')
policy_admin_age = policy_admin_age[['정책ID', '서비스명_x', '소관기관', '소관기관유형.1', '생애주기', '선정기준', '서비스목적', '지원내용', '지원대상', '대상연령시작', '대상연령끝']]

In [7]:
# 생애주기 feature의 카테고리
birth_lst = list()
for sentences in policy_admin_age[policy_admin_age['생애주기'].notnull()]['생애주기'].unique():
    lst = sentences.split('||')
    for i in lst:
      if i not in birth_lst:
        birth_lst.append(i)

birth_lst

['영·유아', '아동·청소년', '청년', '중장년', '노년기']

#### 필터링 사용자 함수

In [8]:
# 생애주기 feature 필터링 코드
def age_default(start_life, end_life):
  if start_life == birth_lst[0] :
    min = 0
  elif start_life == birth_lst[1] :
    min = 6
  elif start_life == birth_lst[2] :
    min = 19
  elif start_life == birth_lst[3] :
    min = 34
  elif start_life == birth_lst[4] :
    min = 65
  
  if end_life == birth_lst[0] :
    max = 6
  elif end_life == birth_lst[1] :
    max = 19
  elif end_life == birth_lst[2] :
    max = 34
  elif end_life == birth_lst[3] :
    max = 70
  elif end_life == birth_lst[4] :
    max = 120
  return min, max


# 지원대상 feature 필터링 코드
def age_sel(text, i):
  if str(text) == 'nan':
    return -1., -1.
  min_lst = []
  max_lst = []

  regex1 = '[0-9]{1,2} ?세 ?이상|[0-9]{1,2} ?세 ?이하|[0-9]{1,2} ?세 ?초과|[0-9]{1,2} ?세 ?미만'
  regex2 = '[0-9]{1,2} ?개월 ?이상|[0-9]{1,2} ?개월 ?이하|[0-9]{1,2} ?개월 ?초과|[0-9]{1,2} ?개월 ?미만'
  regex5 = '[0-9]{1,2} ?대 ?이상|[0-9]{1,2} ?대 ?이하|[0-9]{1,2} ?대 ?초과|[0-9]{1,2} ?대 ?미만'
  regex3 = '[0-9]{1,2}세? ?~ ?만? ?[0-9]{1,2}세' # 3~5세 | 6세~만19세
  regex4 = '[0-9]{1,2}개월? ?~ ?만? ?[0-9]{1,2}개월'

  r = re.compile(f'{regex1}|{regex2}|{regex3}|{regex4}|{regex5}')
  m = r.findall(text)

  if len(m) == 0:
    return -1, -1
  else:
    for i, sel in enumerate(m):
      if '~' in sel:
        a, b = map(lambda x: int(re.search('[0-9]+', x).group()), sel.split('~'))
        min_lst.append(a)
        max_lst.append(b)
        break
      elif ('이상' in sel) or ('초과' in sel):
        if '세' in sel:
          min_lst.append(int(re.search('[0-9]+', sel).group()))
        else:
          age = int(re.search('[0-9]+', sel).group()) / 12
          min_lst.append(round(age))
        continue
      else:
        if '세' in sel:
          max_lst.append(int(re.search('[0-9]+', sel).group()))
        else:
          age = int(re.search('[0-9]+', sel).group()) / 12
          max_lst.append(round(age))
        continue
      
  try:
    minimum = min(min_lst)
  except:
    minimum = 0
  
  try:
    maximum = max(max_lst)
  except:
    maximum = 0

  return minimum, maximum

#### 필터링 코드

In [9]:
%%time

policy_solv_start_age2 = np.array([])
policy_solv_end_age2 = np.array([])

for i in range(len(policy_admin_age)):
  
  # 지원대상 filtering
  a, b = age_sel(policy_admin_age.지원대상.iloc[i] , i)
  if a > 0:
    minimum = a
  if b > 0:
    maximum = b

  # 생애주기 filtering
  # if a < 1 or b < 1:
  life_ages = policy_admin_age.생애주기.iloc[i]
  try:
    if '||' in life_ages:
      start_life = life_ages.split('||')[0]
      end_life = life_ages.split('||')[-1]
    else:
      start_life = end_life = life_ages 
    c, d = age_default(start_life, end_life)
  except:
    c = 0
    d = 120

  if ((a == -1.) and (b == -1.)) or ((a == 0) and (b == 0)):
    minimum = c
    maximum = d 
  elif (a == 0) and (b >0):
    minimum = c
  elif (a > 0 ) and (b == 0):
    maximum = d
  
  # 확인
  if minimum >= maximum :
    minimum = c
    maximum = d

  policy_solv_start_age2 = np.append(policy_solv_start_age2, [minimum])
  policy_solv_end_age2 = np.append(policy_solv_end_age2, [maximum])
  

CPU times: user 21.9 s, sys: 2.81 s, total: 24.8 s
Wall time: 24.8 s


In [10]:
policy_admin_age['대상연령_시작'] = policy_solv_start_age2
policy_admin_age['대상연령_끝'] = policy_solv_end_age2

In [11]:
policy_admin_age[['서비스명_x','생애주기','지원대상', '대상연령_시작', '대상연령_끝']].iloc[1000:2000]

,서비스명_x,생애주기,지원대상,대상연령_시작,대상연령_끝
1000,북한이탈주민 상담 지원,None,○ 상담을 원하는 북한이탈주민,0.0,120.0
1001,북한이탈주민 자립자활 지원,None,○ 북한이탈주민,0.0,120.0
1002,북한이탈주민 주택 알선 지원,None,"○ 북한이탈주민으로 보호 ""또는 비보호"" 결정된 자",0.0,120.0
1003,무주택 전후 납북피해자 주거 지원,None,○ 무주택 전후 납북피해자\n -한국토지주택공사 등에서 납북피해자 대상 주택 배정물...,0.0,120.0
1004,교육복지우선지원,None,"○ 수급자 학생, 교육감이 정하는 차상위 계층 학생, 한 부모 가족 보호 대상 학생...",0.0,120.0
...,...,...,...,...,...
1995,농정시책 교육 지원,청년||중장년||노년기,희망농업인\n\n농축수산인,19.0,120.0
1996,영농기술혁신 교육 지원,청년||중장년||노년기,희망농업인\n\n농축수산인,19.0,120.0
1997,정신건강 증진센터 운영(아동·청소년 정신 보건 사업 및 자살예방사업 포함),아동·청소년||청년||중장년||노년기,○ 정신건강복지센터 이용대상 : 중증정신질환자를 포함한 해당 지역 주민 및 군인\...,6.0,18.0
1998,원격 치매 클리닉 운영 지원,노년기,도내 의료취약지역(9개 군) 치매 안심 센터 등록환자 \n※ 의료취약지역: 정신건...,65.0,120.0


#### 대상연령 맞춘 비율

In [12]:
policy_admin_age['대상연령_시작'] = policy_admin_age['대상연령_시작'].astype(int)
policy_admin_age['대상연령_끝'] = policy_admin_age['대상연령_끝'].astype(int)
policy_admin_age.대상연령_끝 = policy_admin_age.대상연령_끝.replace(120, 99)

In [13]:
# 대상연령시작 맞춘 비율
print(len(policy_admin_age[policy_admin_age.대상연령시작 == policy_admin_age['대상연령_시작']]) / len(policy_admin_age) * 100)

# 대상연령끝 맞춘 비율
print(len(policy_admin_age[policy_admin_age.대상연령끝 == policy_admin_age['대상연령_끝']]) / len(policy_admin_age) * 100)

54.18484279999096
70.6179508622833


#### 필터링 pickle dump

In [14]:
policy_admin_age.columns

Index(['정책ID', '서비스명_x', '소관기관', '소관기관유형.1', '생애주기', '선정기준', '서비스목적', '지원내용',
       '지원대상', '대상연령시작', '대상연령끝', '대상연령_시작', '대상연령_끝'],
      dtype='object')

In [15]:
len(policy), len(policy_admin_age)

(88486, 88486)

In [16]:
policy_age_done = open('/content/drive/MyDrive/웰로/data/policy_age_done_2.pkl', 'wb')
pickle.dump(policy_admin_age,policy_age_done)

### 지역(시도), 지역(시군구)

In [17]:
policy.columns

Index(['정책ID', '정책서비스ID', '서비스명', '소관기관', '소관기관유형', '소관기관.1', '문의처전화번호',
       '소관기관유형.1', '온라인신청사이트URL', '생애주기', '신청절차', '선정기준', '지원유형', '서비스목적',
       '지원내용', '지원대상', '신청기한_시작', '신청기한_끝', '상세조회URL', '태그-신청절차', '태그-자녀',
       '태그-수집유형', '태그-가구원', '태그-성별', '태그-학력', '태그-직장', '태그-결혼', '태그-지원유형',
       '태그-지역(시도)', '태그-관심상황특성', '태그-자녀상세', '태그-직장상세', '태그-지원유형상세',
       '태그-지역(시군구)', '태그-대상특성상세', '태그-대상특성', '태그-대상연령시작', '태그-대상연령끝',
       '태그-중위소득시작', '태그-중위소득끝'],
      dtype='object')

In [18]:
admin.columns

Index(['정책ID', '정책서비스ID', '서비스명', '신청절차', '자녀', '수집유형', '가구원', '성별', '학력',
       '직장', '결혼', '지원유형', '지역(시도)', '관심상황특성', '자녀상세', '직장상세', '지원유형상세',
       '지역(시군구)', '대상특성상세', '대상특성', '대상연령시작', '대상연령끝', '중위소득기준-시작',
       '중위소득기준-종료'],
      dtype='object')

In [19]:
# policy + admin 중 대상연령에 필요한 feature
policy_admin_loca = pd.merge(left=policy, right=admin, how='inner', on='정책ID')
policy_admin_loca = policy_admin_loca[['정책ID', '서비스명_x', '소관기관', '소관기관유형.1', '지원내용', '지원대상', '지역(시도)', '지역(시군구)']]
len(policy_admin_loca), len(policy)

(88486, 88486)

#### city dict

In [20]:
loca_dict = {'전국':[],
             '서울특별시': ['전체', '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'],
             '부산광역시': ['전체', '중구', '서구', '동구', '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구', '금정구', '강서구', '연제구', '수영구', '사상구', '기장군'],
             '대구광역시': ['전체', '중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군'],
             '인천광역시': ['전체', '중구', '동구', '미추홀구', '연수구', '남동구', '부평구', '계양구', '서구', '강화군', '옹진군'],
             '광주광역시': ['전체', '동구', '서구', '남구', '북구', '광산구'],
             '대전광역시': ['전체', '동구', '중구', '서구', '유성구', '대덕구'],
             '울산광역시': ['전체', '중구', '남구', '동구', '북구', '울주군'],
             '세종특별자치시': ['세종시 전체'],
             '경기도': ['전체', '수원시', '성남시', '고양시', '용인시', '부천시', '안산시', '안양시', '남양주시', '화성시', '평택시', '의정부시', '시흥시', '파주시', '광명시', '김포시', '군포시', '광주시', '이천시', '양주시', '오산시', '구리시', '안성시', '포천시', '의왕시', '하남시', '여주시', '양평군', '동두천시', '과천시', '가평군', '연천군'],
             '강원도': ['전체', '춘천시', '원주시', '강릉시', '동해시', '태백시', '속초시', '삼척시', '홍천군', '횡성군', '영월군', '평창군', '정선군', '철원군', '화천군', '양구군', '인제군', '고성군', '양양군'],
             '충청북도': ['전체', '청주시', '충주시', '제천시', '보은군', '옥천군', '영동군', '진천군', '괴산군', '음성군', '단양군', '증평군'],
             '충청남도': ['전체', '천안시', '공주시', '보령시', '아산시', '서산시', '논산시', '계룡시', '당진시', '금산군', '부여군', '서천군', '청양군', '홍성군', '예산군', '태안군'],
             '전라북도': ['전체', '전주시', '군산시', '익산시', '정읍시', '남원시', '김제시', '완주군', '진안군', '무주군', '장수군', '임실군', '순창군', '고창군', '부안군'],
             '전라남도': ['전체', '목포시', '여수시', '순천시', '나주시', '광양시', '담양군', '곡성군', '구례군', '고흥군', '보성군', '화순군', '장흥군', '강진군', '해남군', '영암군', '무안군', '함평군', '영광군', '장성군', '완도군', '진도군', '신안군'],
             '경상북도': ['전체', '포항시', '경주시', '김천시', '안동시', '구미시', '영주시', '영천시', '상주시', '문경시', '경산시', '군위군', '의성군', '청송군', '영양군', '영덕군', '청도군', '고령군', '성주군', '칠곡군', '예천군', '봉화군', '울진군', '울릉군'],
             '경상남도': ['전체', '창원시', '진주시', '통영시', '사천시', '김해시', '밀양시', '거제시', '양산시', '의령군', '함안군', '창녕군', '고성군', '남해군', '하동군', '산청군', '함양군', '거창군', '합천군'],
             '제주특별자치도': ['제주시', '서귀포시']
             }

In [21]:
loca_short_dict = {'전국':[],
             '서울': ['전체', '종로', '중구', '용산', '성동구', '광진구', '동대문', '중랑구', '성북구', '강북구', '도봉', '노원', '은평', '서대문', '마포', '양천', '강서구', '구로구', '금천구', '영등포', '동작구', '관악구', '서초', '강남', '송파', '강동구'],
             '부산': ['전체', '중구', '서구', '동구', '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구', '금정구', '강서구', '연제구', '수영구', '사상구', '기장'],
             '대구': ['전체', '중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성'],
             '인천': ['전체', '중구', '동구', '미추홀구', '연수구', '남동구', '부평구', '계양구', '서구', '강화', '옹진'],
             '광주': ['전체', '동구', '서구', '남구', '북구', '광산구'],
             '대전': ['전체', '동구', '중구', '서구', '유성구', '대덕구'],
             '울산': ['전체', '중구', '남구', '동구', '북구', '울주'],
             '세종': ['전체'],
             '경기': ['전체', '수원', '성남', '고양', '용인', '부천', '안산', '안양', '남양주', '화성', '평택', '의정부', '시흥', '파주', '광명', '김포', '군포', '광주', '이천', '양주', '오산', '구리', '안성', '포천', '의왕', '하남', '여주', '양평', '동두천', '과천', '가평', '연천'],
             '강원': ['전체', '춘천', '원주', '강릉', '동해', '태백', '속초', '삼척', '홍천', '횡성', '영월', '평창', '정선', '철원', '화천', '양구', '인제', '고성', '양양'],
             '충청북': ['전체', '청주', '충주', '제천', '보은', '옥천', '영동', '진천', '괴산', '음성', '단양', '증평'],
             '충청남': ['전체', '천안', '공주', '보령', '아산', '서산', '논산', '계룡', '당진', '금산', '부여', '서천', '청양', '홍성', '예산', '태안'],
             '전라북': ['전체', '전주', '군산', '익산', '정읍', '남원', '김제', '완주', '진안', '무주', '장수', '임실', '순창', '고창', '부안'],
             '전라남': ['전체', '목포', '여수', '순천', '나주', '광양', '담양', '곡성', '구례', '고흥', '보성', '화순', '장흥', '강진', '해남', '영암', '무안', '함평', '영광', '장성', '완도', '진도', '신안'],
             '경상북': ['전체', '포항', '경주', '김천', '안동', '구미', '영주', '영천', '상주', '문경', '경산', '군위', '의성', '청송', '영양', '영덕', '청도', '고령', '성주', '칠곡', '예천', '봉화', '울진', '울릉'],
             '경상남': ['전체', '창원', '진주', '통영', '사천', '김해', '밀양', '거제', '양산', '의령', '함안', '창녕', '고성', '남해', '하동', '산청', '함양', '거창', '합천'],
             '제주': ['제주', '서귀포']
             }

In [22]:
loca_tag_dict = {'A01-00': [],
             'A01-01': ['A01-01-00', 'A01-01-01', 'A01-01-02', 'A01-01-03', 'A01-01-04', 'A01-01-05', 'A01-01-06', 'A01-01-07', 'A01-01-08', 'A01-01-09', 'A01-01-10', 'A01-01-11', 'A01-01-12', 'A01-01-13', 'A01-01-14', 'A01-01-15', 'A01-01-16', 'A01-01-17', 'A01-01-18', 'A01-01-19', 'A01-01-20', 'A01-01-21', 'A01-01-22', 'A01-01-23', 'A01-01-24', 'A01-01-25'],
             'A01-02': ['A01-02-00', 'A01-02-01', 'A01-02-02', 'A01-02-03', 'A01-02-04', 'A01-02-05', 'A01-02-06', 'A01-02-07', 'A01-02-08', 'A01-02-09', 'A01-02-10', 'A01-02-11', 'A01-02-12', 'A01-02-13', 'A01-02-14', 'A01-02-15', 'A01-02-16'],
             'A01-03': ['A01-03-00', 'A01-03-01', 'A01-03-02', 'A01-03-03', 'A01-03-04', 'A01-03-05', 'A01-03-06', 'A01-03-07', 'A01-03-08'],
             'A01-04': ['A01-04-00', 'A01-04-01', 'A01-04-02', 'A01-04-03', 'A01-04-04', 'A01-04-05', 'A01-04-06', 'A01-04-07', 'A01-04-08', 'A01-04-09', 'A01-04-10'],
             'A01-05': ['A01-05-00', 'A01-05-01', 'A01-05-02', 'A01-05-03', 'A01-05-04', 'A01-05-05'],
             'A01-06': ['A01-06-00', 'A01-06-01', 'A01-06-02', 'A01-06-03', 'A01-06-04', 'A01-06-05'],
             'A01-07': ['A01-07-00', 'A01-07-01', 'A01-07-02', 'A01-07-03', 'A01-07-04', 'A01-07-05'],
             'A01-08': ['A01-08-00'],
             'A01-09': ['A01-09-00', 'A01-09-01', 'A01-09-02', 'A01-09-03', 'A01-09-04', 'A01-09-05', 'A01-09-06', 'A01-09-07', 'A01-09-08', 'A01-09-09', 'A01-09-10', 'A01-09-11', 'A01-09-12', 'A01-09-13', 'A01-09-14', 'A01-09-15', 'A01-09-16', 'A01-09-17', 'A01-09-18', 'A01-09-19', 'A01-09-20', 'A01-09-21', 'A01-09-22', 'A01-09-23', 'A01-09-24', 'A01-09-25', 'A01-09-26', 'A01-09-27', 'A01-09-28', 'A01-09-29', 'A01-09-30', 'A01-09-31'],
             'A01-10': ['A01-10-00', 'A01-10-01', 'A01-10-02', 'A01-10-03', 'A01-10-04', 'A01-10-05', 'A01-10-06', 'A01-10-07', 'A01-10-08', 'A01-10-09', 'A01-10-10', 'A01-10-11', 'A01-10-12', 'A01-10-13', 'A01-10-14', 'A01-10-15', 'A01-10-16', 'A01-10-17', 'A01-10-18'],
             'A01-11': ['A01-11-00', 'A01-11-01', 'A01-11-02', 'A01-11-03', 'A01-11-04', 'A01-11-05', 'A01-11-06', 'A01-11-07', 'A01-11-08', 'A01-11-09', 'A01-11-10', 'A01-11-11'],
             'A01-12': ['A01-12-00', 'A01-12-01', 'A01-12-02', 'A01-12-03', 'A01-12-04', 'A01-12-05', 'A01-12-06', 'A01-12-07', 'A01-12-08', 'A01-12-09', 'A01-12-10', 'A01-12-11', 'A01-12-12', 'A01-12-13', 'A01-12-14', 'A01-12-15'],
             'A01-13': ['A01-13-00', 'A01-13-01', 'A01-13-02', 'A01-13-03', 'A01-13-04', 'A01-13-05', 'A01-13-06', 'A01-13-07', 'A01-13-08', 'A01-13-09', 'A01-13-10', 'A01-13-11', 'A01-13-12', 'A01-13-13', 'A01-13-14'],
             'A01-14': ['A01-14-00', 'A01-14-01', 'A01-14-02', 'A01-14-03', 'A01-14-04', 'A01-14-05', 'A01-14-06', 'A01-14-07', 'A01-14-08', 'A01-14-09', 'A01-14-10', 'A01-14-11', 'A01-14-12', 'A01-14-13', 'A01-14-14', 'A01-14-15', 'A01-14-16', 'A01-14-17', 'A01-14-18', 'A01-14-19', 'A01-14-20', 'A01-14-21', 'A01-14-22'],
             'A01-15': ['A01-15-00', 'A01-15-01', 'A01-15-02', 'A01-15-03', 'A01-15-04', 'A01-15-05', 'A01-15-06', 'A01-15-07', 'A01-15-08', 'A01-15-09', 'A01-15-10', 'A01-15-11', 'A01-15-12', 'A01-15-13', 'A01-15-14', 'A01-15-15', 'A01-15-16', 'A01-15-17', 'A01-15-18', 'A01-15-19', 'A01-15-20', 'A01-15-21', 'A01-15-22', 'A01-15-23'],
             'A01-16': ['A01-16-00', 'A01-16-01', 'A01-16-02', 'A01-16-03', 'A01-16-04', 'A01-16-05', 'A01-16-06', 'A01-16-07', 'A01-16-08', 'A01-16-09', 'A01-16-10', 'A01-16-11', 'A01-16-12', 'A01-16-13', 'A01-16-14', 'A01-16-15', 'A01-16-16', 'A01-16-17', 'A01-16-18'],
             'A01-17': ['A01-17-00', 'A01-17-01']
             }

#### 필터링 코드
- 1차 필터링: 소관기관 유형이 시군구 or 광역시도일 경우

- 2차 필터링: 소관기관에 지역명이 있는 경우

In [23]:
%%time

city_lst = []
area_lst = []

city_list = list(loca_dict.keys())
city_short_dict_list = list(loca_short_dict.keys())

for i in range(len(policy_admin_loca)):
  # ciyt, area를  None으로 초기화
  city, area = None, None

  # 소관기관유형이 시군구 or  광역시도라면 채울 수 있음.
  if policy_admin_loca['소관기관유형.1'].iloc[i] == '시군구':
    city, area = policy_admin_loca.소관기관.iloc[i].split()
  elif policy_admin_loca['소관기관유형.1'].iloc[i] == '광역시도':
    city = policy_admin_loca.소관기관.iloc[i]
    area = loca_dict[city][0]
  

  # 소관기관유형이 시군구 or  광역시가 아닐 경우, 소관기관에 이름이 들어있으면 채울 수 있음.
  if city == None:
    text = policy_admin_loca.소관기관.iloc[i]
    text_city = ''
    text_area = ''
    c = False
    #city 한 번 싹 돌기
    for c_idx, city in enumerate(city_short_dict_list):
      city_real = city_list[c_idx] # 진짜 city 이름

      if c == True:
        break

      if city in text:
        text_city += f'{city_real},'
        c = True

      # area 한 번 싹 돌기
      for a_idx, area in enumerate(loca_short_dict[city]):
        if area in text:
          area_real = loca_dict[city_real][a_idx]
          text_area += f'{area_real},'
          # area 기준 city 삽입
          if city_real not in text_city:
            text_city += f'{city_real},'
    
    if len(text_city) == 0 and len(text_area) == 0:
      city = '전국'
      area = None
    elif text_city.count(',') == 1 and text_city.count(',') == 0:
      city = text_city[:-1]
      area = loca_dict[city][0]
    elif len(text_city) == 0 and text_area.count(',') == 1:
      city = None
      area = text_area[:-1]
    else:
      city = text_city[:-1]
      area = text_area[:-1]

  city_lst.append(city)
  area_lst.append(area)

CPU times: user 2.66 s, sys: 19.9 ms, total: 2.68 s
Wall time: 2.68 s


In [24]:
len(city_lst), len(area_lst), len(policy_admin_loca), len(admin)

(88486, 88486, 88486, 93027)

In [25]:
policy_admin_loca['필터링_시도'] = city_lst
policy_admin_loca['필터링_시군구'] = area_lst

In [26]:
# 시도 정확히 맞춘 비율
print(len(policy_admin_loca[policy_admin_loca['지역(시도)'] == policy_admin_loca['필터링_시도']]) / len(policy_admin_loca) * 100)

# 시군구 정확히 맞춘 비율
print(len(policy_admin_loca[policy_admin_loca['지역(시군구)'] == policy_admin_loca['필터링_시군구']]) / len(policy_admin_loca) * 100)

99.6236692810162
83.42788689736228


#### 지역 맞춘 비율

In [27]:
# 지역(시도)
print(len(policy_admin_loca[policy_admin_loca['지역(시도)'] == policy_admin_loca['필터링_시도']]) / len(policy_admin_loca) * 100)

# 지역(시군구)
print(len(policy_admin_loca[policy_admin_loca['지역(시군구)'] == policy_admin_loca['필터링_시군구']]) / len(policy_admin_loca) * 100)

99.6236692810162
83.42788689736228


#### 필터링 pickle dump

In [28]:
policy_loca_done = open('/content/drive/MyDrive/웰로/data/policy_loca_done_2.pkl', 'wb')
pickle.dump(policy_admin_loca,policy_loca_done)

### 대상특성

In [29]:
# policy + admin 중 대상연령에 필요한 feature
policy_admin_cha = pd.merge(left=policy, right=admin, how='inner', on='정책ID')
policy_admin_cha = policy_admin_cha[['정책ID', '서비스명_x', '선정기준', '서비스목적', '지원내용', '지원대상', '대상특성', '대상특성상세']]

In [30]:
policy_admin_cha = policy_admin_cha.sort_values('대상특성')
policy_admin_cha.fillna('', inplace=True)
policy_admin_cha['text'] = '◈서비스명_x◈' + policy_admin_cha['서비스명_x'] + '◈선정기준◈' + policy_admin_cha['선정기준'] + '◈서비스목적◈' + policy_admin_cha['서비스목적'] + '◈지원내용◈' + policy_admin_cha['지원내용'] + '◈지원대상◈' +policy_admin_cha['지원대상']
policy_admin_cha.head()

,정책ID,서비스명_x,선정기준,서비스목적,지원내용,지원대상,대상특성,대상특성상세,text
11043,11042,"저소득층 자산형성 지원(희망, 내일키움통장 사업)",○희망키움통장Ⅰ \n ① 일하는 생계·의료 수급가구\n ② 가구 전체의 총 ...,○ 희망키움통장Ⅰ: 일하는 생계·의료수급가구의 탈수급 지원\n○ 희망키움통장Ⅱ: 일...,"○ 희망키움통장 Ⅰ: 본인 저축액 월 10만원, 정부 지원금 월 평균 44만원(3년...",○ 희망키움통장 Ⅰ: 총 근로(사업)소득이 기준 중위소득 40%의 60%~기준 중위...,1인가구,,"◈서비스명_x◈저소득층 자산형성 지원(희망, 내일키움통장 사업)◈선정기준◈○희망키움..."
58727,58728,여성 안심 택배함 설치 운영,,"혼자 있는 여성이 집에서 물건 받기를 원치 않거나, 맞벌이 가정처럼 집에서 택배를 ...",1호점 : 무선로 190(선원동) 여천동 주민센터 실내주차장 안심 택배함\n여수시 ...,시민(1인 가구 와 맞벌이 가구 등 모든 가구)\n\n여성||일반,1인가구,,◈서비스명_x◈여성 안심 택배함 설치 운영◈선정기준◈◈서비스목적◈혼자 있는 여성이 ...
69948,69949,"저소득층 자산형성 지원(희망, 내일키움통장 사업)",○희망키움통장Ⅰ \n ① 일하는 생계·의료 수급가구\n ② 가구 전체의 총 ...,○ 희망키움통장Ⅰ: 일하는 생계·의료수급가구의 탈수급 지원\n○ 희망키움통장Ⅱ: 일...,"○ 희망키움통장 Ⅰ: 본인 저축액 월 10만원, 정부 지원금 월 평균 33만원(3년...",○ 희망키움통장 Ⅰ: 총 근로(사업)소득이 기준 중위소득 40%의 60%~기준 중위...,1인가구,,"◈서비스명_x◈저소득층 자산형성 지원(희망, 내일키움통장 사업)◈선정기준◈○희망키움..."
29502,29501,"저소득층 자산형성 지원(희망, 내일키움통장 사업)",○희망키움통장Ⅰ \n ① 일하는 생계·의료 수급가구\n ② 가구 전체의 총 ...,○ 희망키움통장Ⅰ: 일하는 생계·의료수급가구의 탈수급 지원\n○ 희망키움통장Ⅱ: 일...,"○ 희망키움통장 Ⅰ: 본인 저축액 월 10만원, 정부 지원금 월 평균 33만원(3년...",○ 희망키움통장 Ⅰ: 총 근로(사업)소득이 기준 중위소득 40%의 60%~기준 중위...,1인가구,,"◈서비스명_x◈저소득층 자산형성 지원(희망, 내일키움통장 사업)◈선정기준◈○희망키움..."
72567,72568,"저소득층 자산형성 지원(희망, 내일키움통장 사업)",○희망키움통장Ⅰ \n ① 일하는 생계·의료 수급가구\n ② 가구 전체의 총 ...,○ 희망키움통장Ⅰ: 일하는 생계·의료수급가구의 탈수급 지원\n○ 희망키움통장Ⅱ: 일...,"○ 희망키움통장 Ⅰ: 본인 저축액 월 10만원, 정부 지원금 월 평균 36만원(3년...",○ 희망키움통장 Ⅰ: 총 근로(사업)소득이 기준 중위소득 40%의 60%~기준 중위...,1인가구,,"◈서비스명_x◈저소득층 자산형성 지원(희망, 내일키움통장 사업)◈선정기준◈○희망키움..."


In [31]:
print(policy_admin_cha[policy_admin_cha['text'].str.contains('영유아')]['text'].iloc[1])

◈서비스명_x◈의료급여수급자 만 6세 미만 영유아 건강검진 지원◈선정기준◈만 6세 미만 영유아(의료급여수급권자)◈서비스목적◈영유아의 건강증진을 도모하고 건강한 미래 인적자본으로 성장하도록 지원 및 영유아 연령에 적합한 건강검진 프로그램을 도입하여 영유아의 성장/발달 사항을 추적 관리하고 보호자에게 적절한 교육 프로그램을 제공하기 위함◈지원내용◈○ 영유아 건강검진 서비스
 - 주요 선별 목표질환: 성장 이상, 발달 이상, 비만, 안전사고, 영아 급사 증후군, 청각이상, 시각 이상, 치아우식증 등
 - 영유아 건강검진 항목 : 각 월령에 특화된 문진(시각·청각 문진 포함)과 진찰, 신체계측(신장·체중·두위)이 공통 실시, 9종의 건강교육과 발달평가 및 상담(4개월 제외)으로 구성
 - 검진 주기: 4개월, 9개월, 18개월, 30개월, 42개월, 54개월, 66개월



건강·의료·사망>의료지원>저소득층||결혼·육아·교육>보육/교육지원>건강검진/예방접종◈지원대상◈만 6세 미만 영유아(의료급여수급권자)

저소득층


In [32]:
policy_admin_cha.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88486 entries, 11043 to 88295
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   정책ID    88486 non-null  int64 
 1   서비스명_x  88486 non-null  object
 2   선정기준    88486 non-null  object
 3   서비스목적   88486 non-null  object
 4   지원내용    88486 non-null  object
 5   지원대상    88486 non-null  object
 6   대상특성    88486 non-null  object
 7   대상특성상세  88486 non-null  object
 8   text    88486 non-null  object
dtypes: int64(1), object(8)
memory usage: 6.8+ MB


#### 필터링 코드

In [33]:
%%time
# 대상특성 만들어보자
tar_cha = []
tar_cha_detail = []
for i in range(len(policy_admin_cha)):
    cha = []
    detail = []
    text = policy_admin_cha.text.iloc[i]

    # 장애인
    if text.count('장애') >= 3:
        cha.append('장애인')
        # 하위태그
        if text.count('심한|중증') >= 1:
          detail.append('심한 장애')
        elif text.count('심하지|경증') >= 1:
          detail.append('심하지 않은 장애')

    # 농축수산인
    cha_words = ['농업', '농축', '어업' '원예', '축산', '임업']
    if sum(text.count(x) for x in cha_words) >= 2: 
        cha.append('농축수산인')
        # 하위태그 - 경관농업인
        if ('농업' in text) & ('경관' in text) & ('경관' in text) & (text.count('축산') < 2): # 빼야함 --------------
            detail.append('경관농업인')
        # 하위태그 - 경종농업인
        if ('농업' in text) & (sum(text.count(x) for x in ['경종', '농업인', '농산물', '재배', '귀농', '영농']) >= 4) & (text.count('축산') < 2):
            detail.append('경관농업인')
        # 하위태그 - 과수농업인
        if ('농업' in text) & (sum(text.count(x) for x in ['과수', '과일', '유기농']) >= 2) & (text.count('축산') < 2):
            detail.append('과수농업인')
        # 하위태그 - 원예·특용농업인
        if (sum(text.count(x) for x in ['화훼', '원예', '특용', '작물']) >= 3) & (text.count('축산') < 2):
            detail.append('원예·특용농업인')
        # 하위태그 - 농지소유자
        if sum(text.count(x) for x in ['토지', '토양', '농지', '소유']) >= 4:
            detail.append('농지소유자')

        # 하위태그 - 연근해-구획어업인
        if (('어업' in text) or ('수산' in text)) & (sum(text.count(x) for x in ['연근해', '구획', '어선']) >= 1):
            detail.append('연근해-구획어업인')
        # 하위태그 - 양식업인
        if (('어업' in text) or ('수산' in text)) & ('양식' in text):
            detail.append('양식업인')
        # 하위태그 - 어선소유자
        if ('어업' in text) & (sum(text.count(x) for x in ['어선', '소유', '선박']) >= 2):
            detail.append('어선소유자')
        
        # 하위태그 - 축산업인
        if ('축산' in text) & (sum(text.count(x) for x in ['가축', '분뇨', '퇴비']) >= 1):
            detail.append('축산업인')
        
        # 하위태그 - 임업인
        if (('임업' in text) or ('산림' in text)) & (sum(text.count(x) for x in ['임업', '산림', '숲']) >= 2):
            detail.append('임업인')


        # 하위태그 - 기타농업인, 기타어업인
        if len(detail) == 0:
            if '농업' in text:
                detail.append('기타농업인')
            if '어업' in text:
                detail.append('기타어업인')
            # else:
            #     detail.append(None)

    # 질병/부상/질환자
    cha_words = ['환자', '입원', '질병', '부상']
    if sum(text.count(x) for x in cha_words) >= 3:
        cha.append('질병/부상/질환자')
        # 하위태그 - 중증·난치·희귀질환자
        if sum(text.count(x) for x in ['중증', '격리', '난치', '희귀', '암']) >= 2:
            detail.append('중증·난치·희귀질환자')
        # 하위태그 - 요양, 치매환자
        if (sum(text.count(x) for x in ['치매', '요양']) >= 2) & (text.count('군인') < 2):
            detail.append('요양, 치매환자')

    # 난임
    if text.count('난임') >= 2:
        cha.append('난임')

    # 임신부
    if sum(text.count(x) for x in ['임신', '임산부', '출산']) >= 3:
        cha.append('임신부')

    # 출산후
    if sum(text.count(x) for x in ['출산', '산모', '영유아', '영아', '태아', '신생아', '모유', '분만']) >= 3:
        cha.append('출산후')

    # 국가유공자
    if sum(text.count(x) for x in ['유공자' '독립', '보훈', '순국선열', '애국지사']) >= 3:
        cha.append('국가유공자')

    # 제대군인
    if text.count('제대') >= 2:
        cha.append('제대군인')

    # 다문화가족
    if text.count('다문화') >= 3:
        cha.append('다문화가족')

    # 북한이탈주민
    if text.count('북한이탈') >= 2:
        cha.append('북한이탈주민')

    # 한부모가정/조손가정
    if sum(text.count(x) for x in ['한부모', '조손']) >= 4:
        cha.append('한부모가정/조손가정')

    # 1인가구
    if sum(text.count(x) for x in ['1인가구', '1인 가구']) >= 2:
        cha.append('1인가구')

    # 다자녀가정
    if text.count('다자녀') >= 2:
        cha.append('다자녀가정')

    # 독거노인
    if ('독거노인' in text) & (text.count('노인') >= 3):
        cha.append('독거노인')

    # 입양가정
    if text.count('입양가정') >= 2:
        cha.append('입양가정')

    # 외국인/재외국인
    if text.count('외국인') >= 3:
        cha.append('외국인/재외국인')

    # 한센인
    if text.count('한센인') >= 3:
        cha.append('한센인')
    
    # 이산가족
    if '이산가족' in text:
        cha.append('이산가족')

    # 기관/시설
    if sum(text.count(x) for x in ['기괸', '시설']) >= 4:
        cha.append('기관/시설')


    # 해당사항없음
    if (len(cha) == 0) & (len(detail) == 0) :
        cha.append('해당사항없음')
    
    # 해당데이터 대상특성, 대상특성상세 입력
    tar_cha.append(', '.join(cha))
    tar_cha_detail.append(', '.join(detail))

CPU times: user 4.91 s, sys: 15.3 ms, total: 4.92 s
Wall time: 4.94 s


In [34]:
policy_admin_cha['필터링_대상특성'] = tar_cha
policy_admin_cha['필터링_대상특성상세'] = tar_cha_detail

#### 대상특성 맞춘 비율

In [35]:
for i in range(len(policy_admin_cha)):
  if policy_admin_cha.대상특성.iloc[i] == '':
    policy_admin_cha.대상특성.iloc[i] = '해당사항없음'
  
  if policy_admin_cha.필터링_대상특성.iloc[i] == '':
    policy_admin_cha.필터링_대상특성.iloc[i] = '해당사항없음'

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [36]:
# 대상특성 맞춘 비율
print(len(policy_admin_cha[policy_admin_cha.대상특성 == policy_admin_cha.필터링_대상특성]) / len(policy_admin_cha) * 100)

# 대상특성상세 맞춘 비율
print(len(policy_admin_cha[policy_admin_cha.대상특성상세 == policy_admin_cha.필터링_대상특성상세]) / len(policy_admin_cha) * 100)

42.874579029450985
68.45602694211513


#### 필터링 pickle dump

In [37]:
policy_target_done2 = open('/content/drive/MyDrive/웰로/data/policy_target_done2.pkl', 'wb')
pickle.dump(policy_admin_cha,policy_target_done2)

In [38]:
a = pd.read_pickle('/content/drive/MyDrive/웰로/data/policy_target_done2.pkl')
a.columns

EOFError: ignored

In [ ]:
a[a['필터링_대상특성'].str.contains('농축수산인')].iloc[:3]

In [ ]:
for i in range(3):
  data = a[a['필터링_대상특성'].str.contains('농축수산인')].iloc[i]
  print('=======================')
  for name in a.columns:
    print(f'◈{name}◈')
    print(data[name])
    print()
  print()

### 관심상황특성

In [39]:
# policy + admin 중 대상연령에 필요한 feature
policy_admin_situation = pd.merge(left=policy, right=admin, how='inner', on='정책ID')
policy_admin_situation = policy_admin_situation[['정책ID', '서비스명_x', '소관기관', '선정기준', '서비스목적', '지원내용', '지원대상', '관심상황특성']]

policy_admin_situation['text_A13'] = '◈서비스명_x◈' + policy_admin_situation['서비스명_x'] + '◈소관기관◈' + policy_admin_situation['소관기관'] +'◈선정기준◈' + policy_admin_situation['선정기준'] + '◈서비스목적◈' + policy_admin_situation['서비스목적'] + '◈지원내용◈' + policy_admin_situation['지원내용'] + '◈지원대상◈' +policy_admin_situation['지원대상']
policy_admin_situation.head()

,정책ID,서비스명_x,소관기관,선정기준,서비스목적,지원내용,지원대상,관심상황특성,text_A13
0,1,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),교육부,1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...,누리과정이란 만 3~5세를 대상으로 하는 어린이집 표준보육과정과 유치원 교육과정을 ...,"만 3~5세 누리과정 운영에 필요한 비용 지원 \n - 보육료, 담당교사 처우 개선...",지원대상 : 어린이집을 이용하는 만 3~5세 아동\n- 취학유예 : 만 6세아 초등...,None,◈서비스명_x◈만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영)◈소관기관...
1,2,근로·자녀장려금,국세청,"○ 근로소득, 종교인소득 또는 종교인소득이 있고 아래 요건을 모두 충족하는 자\n\...",○ 근로자녀장려세제\n - 저소득계층의 빈곤 탈출을 지원하고 사회안전망을 확충하기 ...,○ 전년도 연간 부부합산 총 급여액 등(사업소득과 근로소득의 합계)에 따라\n - ...,"○ 전년도에 정해진 범위의 근로소득, 사업소득 또는 종교인소득이 있고 근로장려금과 ...",None,"◈서비스명_x◈근로·자녀장려금◈소관기관◈국세청◈선정기준◈○ 근로소득, 종교인소득 또..."
2,3,월세 자금보증,금융위원회,○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...,저소득층 등에게 월세자금보증을 하여 저소득층의 주거 안정성 제고,○ 주택도시기금 주거안정 월세대출 요건을 충족하는 대상자에 대해 월세자금보증 지원\...,○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,None,◈서비스명_x◈월세 자금보증◈소관기관◈금융위원회◈선정기준◈○ (주택도시기금 주거안정...
3,4,‘힘내라 대한민국’특별운영자금 (산업은행),금융위원회,○ 산업은행 심사기준 충족 업체,코로나19로 피해를 입은 중소ㆍ중견기업 지원 프로그램,○ 자금용도 : 운영자금\n\n○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부...,"○ 지원대상 : 코로나19 등 질병, 자연재해 등 국가 재난 발생으로 피해를 입은 ...",None,◈서비스명_x◈‘힘내라 대한민국’특별운영자금 (산업은행)◈소관기관◈금융위원회◈선정기...
4,5,친환경 에너지 절감 장비 보급,해양수산부,"○ 기관ㆍ장비ㆍ설비사업의 사업자 선정은 다음의 순위로 하되, 동일 순위자의 세부 우...",연근해 어업인을 대상으로 노후기관ㆍ장비ㆍ설비 대체 보급으로 안정적 조업기반 조성,○ 유류비 절감을 위한 유류절감장치 설치\n○ 저효율(육상기관 포함)기관 및 사...,"○ 어선 노후기관, 장비를 교체하고자 하는 연근해 어업인",None,◈서비스명_x◈친환경 에너지 절감 장비 보급◈소관기관◈해양수산부◈선정기준◈○ 기관ㆍ...


In [40]:
policy_admin_situation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88486 entries, 0 to 88485
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   정책ID      88486 non-null  int64 
 1   서비스명_x    88486 non-null  object
 2   소관기관      88486 non-null  object
 3   선정기준      51860 non-null  object
 4   서비스목적     88486 non-null  object
 5   지원내용      88486 non-null  object
 6   지원대상      88486 non-null  object
 7   관심상황특성    18697 non-null  object
 8   text_A13  51860 non-null  object
dtypes: int64(1), object(8)
memory usage: 6.8+ MB


In [41]:
# 개인금융지원, 개인교육지원이 가장 많음
policy_admin_situation[policy_admin_situation.관심상황특성.notnull()].관심상황특성.value_counts()

개인금융지원                    6878
개인교육지원                    3716
기업금융지원                    3512
기업교육지원                    1958
개인금융지원, 개인교육지원            1056
결혼예정                       738
개인금융지원, 기업금융지원             226
기업교육지원, 기업금융지원             154
개인교육지원, 개인금융지원              76
개인교육지원, 기업금융지원              70
은퇴예정                        68
개인금융지원, 결혼예정                56
결혼예정, 개인금융지원                36
개인금융지원, null                27
개인교육지원, 기업교육지원              20
입대예정                        12
개인금융지원, 은퇴예정                10
기업교육지원, 개인교육지원               9
결혼예정, 개인교육지원                 9
기업교육지원, 개인교육지원, 기업금융지원       8
개인금융지원, 기업교육지원               7
기업금융지원, 개인금융지원               6
기업교육지원, 기업금융지원, 개인교육지원       6
결혼예정, 기업금융지원                 5
기업금융지원, 개인교육지원               5
개인교육지원, 결혼예정                 3
기업교육지원, 개인금융지원               3
개인금융지원, 개인교육지원, 기업금융지원       3
입대예정, 결혼예정                   3
은퇴예정, 개인금융지원                 2
개인금융지원, 기업금융지원, 개인교육지원       2
결혼예정, 기업교육지원                 2
개인교육지원, 

#### 필터링 코드

In [42]:
policy_admin_situation.fillna('', inplace=True)

In [43]:
policy_admin_situation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88486 entries, 0 to 88485
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   정책ID      88486 non-null  int64 
 1   서비스명_x    88486 non-null  object
 2   소관기관      88486 non-null  object
 3   선정기준      88486 non-null  object
 4   서비스목적     88486 non-null  object
 5   지원내용      88486 non-null  object
 6   지원대상      88486 non-null  object
 7   관심상황특성    88486 non-null  object
 8   text_A13  88486 non-null  object
dtypes: int64(1), object(8)
memory usage: 6.8+ MB


In [44]:
%%time

situation_tag = []
for i in range(len(policy_admin_situation)):
    tag = []
    text = policy_admin_situation.text_A13.iloc[i]

    if len(text) > 0:

      # 개인금융지원
      if (sum(text.count(x) for x in ['금융지원', '자금지원', '생활지원', '금융 지원', '자금 지원', '생활 지원']) >= 1) & (sum(text.count(x) for x in ['자원', '연금', '금리', '금융']) >= 8):
          tag.append('개인금융지원')

      # 개인교육지원
      if ('교육' in text) & (sum(text.count(x) for x in ['지원', '교육']) >= 10):
          tag.append('개인교육지원')

      # 기업금융지원
      if sum(text.count(x) for x in ['경영', '창업', '영세', '장영업', '소상공인', '사업', '기업']) >= 6:
          tag.append('기업금융지원')

      # 기업교육지원
      if ('교육' in text) & (sum(text.count(x) for x in ['경영', '창업', '영세', '장영업', '소상공인', '사업', '기업']) >= 9):
          tag.append('기업교육지원')

      # 입대예정
      if '군입대' in text:
          tag.append('입대예정')

      # 결혼예정
      if (sum(text.count(x) for x in ['결혼 예정', '결혼예정', '혼인 예정', '혼인예정']) >= 1) or (('결혼' in text) & (sum(text.count(x) for x in ['결혼', '혼인', '예정', '출산']) >= 10)):
          tag.append('결혼예정')

      # 은퇴 예정
      if '은퇴' in text:
          tag.append('은퇴예정')
      
      # 폭력/범죄/중독지원
      if sum(text.count(x) for x in ['폭력', '범죄', '중독']) >= 3:
          tag.append('폭력/범죄/중독지원')

      # 자동차/교통지원
      if (sum(text.count(x) for x in ['자동차', '교통', '차량']) >= 3) & ('주택' not in text):
          tag.append('자동차/교통지원')
    
    situation_tag.append(', '.join(tag))

CPU times: user 3.12 s, sys: 5.42 ms, total: 3.13 s
Wall time: 3.15 s


In [45]:
len(situation_tag)

88486

In [46]:
policy_admin_situation['필터링_관심상황특성'] = situation_tag

#### 관심상황특성 맞춘 비율

In [47]:
for i in range(len(policy_admin_situation)):
  if policy_admin_situation.관심상황특성.iloc[i] == 'null':
    policy_admin_situation.관심상황특성.iloc[i] = ''

In [48]:
# 관심상황특성
print(len(policy_admin_situation[policy_admin_situation['관심상황특성'] == policy_admin_situation['필터링_관심상황특성']]) / len(policy_admin_situation) * 100)

65.68157674660398


#### 필터링 pickle dump

In [49]:
policy_situation_done2 = open('/content/drive/MyDrive/웰로/data/policy_situation_done2.pkl', 'wb')
pickle.dump(policy_admin_situation,policy_situation_done2)

### 중위소득

In [50]:
admin.columns

Index(['정책ID', '정책서비스ID', '서비스명', '신청절차', '자녀', '수집유형', '가구원', '성별', '학력',
       '직장', '결혼', '지원유형', '지역(시도)', '관심상황특성', '자녀상세', '직장상세', '지원유형상세',
       '지역(시군구)', '대상특성상세', '대상특성', '대상연령시작', '대상연령끝', '중위소득기준-시작',
       '중위소득기준-종료'],
      dtype='object')

In [51]:
# policy + admin 중 중위소득에 필요한 feature
policy_admin_money = pd.merge(left=policy, right=admin, how='inner', on='정책ID')
policy_admin_money = policy_admin_money[['정책ID', '지원대상', '선정기준', '중위소득기준-시작', '중위소득기준-종료']]
policy_admin_money.fillna('', inplace=True)

#### 필터링 코드

In [52]:
def string_to_money(string):
  nums = re.findall('[0-9][0-9]+', string)
  if len(nums) == 0:
    return None
  else:
    if '분의' in string:
      if len(nums) >= 3:
        try:
          nums = re.findall('[0-9][0-9]+', string.split('%')[0])
          num = (round((int(nums[0])/int(nums[1])*(int(nums[2])/100)) * 100) // 10) * 10
        except:
          try:
            num = (round(int(nums[0])/int(nums[1])*100) // 10) * 10
          except:
            num = 0
      elif len(nums) == 2:
        num = (round(int(nums[0])/int(nums[1])*100) // 10) * 10
      else:
        num = 0
      
      if '이하' in string or '미만' in string:
        range_lst = list(range(0,num+10,10))
      elif '이상' in string or '초과' in string:
        range_lst = list(range(num, 210, 10))
      else:
        range_lst = [num]
    

    elif (('이상' in string) or ('초과' in string)) and (('이하' in string) or ('미만' in string)):
      if len(nums) == 2:
        a = (int(nums[0])//10) * 10
        b = (int(nums[1])//10) * 10
        if a <= b:
          range_lst = list(range(a, b+10, 10))
        else:
          return None
      else:
        return None

    elif ('이상' in string) or ('초과' in string):
      a = (int(nums[0])//10) * 10
      range_lst = list(range(a, 210, 10))

    elif ('이하' in string) or ('미만' in string):
      b = (int(nums[0])//10) * 10
      range_lst = list(range(0, b+10, 10))
    else:
      return None
  return ",".join(map(str, range_lst))
  
money_lst = []

for i in range(len(policy_admin_money)):
  text = policy_admin_money.지원대상.iloc[i] + '\n' + policy_admin_money.선정기준.iloc[i]
  money = []
  if len(text) != 0:
    if '중위' in text:
      tokens = text.split('\n')
      for sentence in tokens:
        if '중위' in sentence:
          strings = sentence.split('중위')
          for string in strings[1:]:
            money_range = string_to_money(string)
            money.append(money_range)
        if '기초생활수급자' in sentence:
          if ('기초생활수급자를 제외' in sentence) or ('기초생활수급자가 아닌' in sentence):
            pass
          else:
            money.append('기초생활수급자')
  else:
    money = None

  if money == None:
    money_lst.append(None)
  else:
    money = list(set(money))
    final = ','.join(item for item in money if item)
    money_lst.append(final)

In [53]:
len(money_lst), len(policy_admin_money)

(88486, 88486)

In [54]:
policy_admin_money['중위소득'] = money_lst

필터링 후 정리

In [55]:
money_new = []
money_lst = policy_admin_money['중위소득']
for i in range(len(policy_admin_money)):
  money = money_lst[i]
  if (money == '') or (money == None):
    money_new.append(None)
  else:
    money = money.split(',')
    ans = []
    if '기초생활수급자' in money:
      money.remove('기초생활수급자')
      ans =  sorted([int(x) for x in set(money) if int(x) <= 200])
      ans = [str(x) for x in ans]
      ans.append('기초생활수급자')
    else:
      ans =  sorted([int(x) for x in set(money) if int(x) <= 200])
      ans = [str(x) for x in ans]
    
    if len(ans) == 0:
      money_new.append(None)
    else:
      money_new.append(','.join(ans))

In [56]:
policy_admin_money['중위소득'] = money_new
policy_admin_money.중위소득.value_counts()

0,10,20,30,40                                                                       1627
0,10,20,30,40,50                                                                    1626
0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200            1311
0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180                     895
0,10,20,30,40,50,60,70,80,90,100,110,120                                             817
0,10,20,30,40,50,60                                                                  772
0,10,20,30,40,50,60,70,80                                                            659
0,10,20,30,40,50,60,70                                                               629
0,10,20,30,40,50,60,70,80,90,100                                                     432
0,10,20,30,40,50,60,70,80,90,100,110,120,130,140                                     186
0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150                                 174
50,60,70,80,90,100   

#### 필터링 pickle dump

In [57]:
policy_money_done = open('/content/drive/MyDrive/웰로/data/policy_money_done.pkl', 'wb')
pickle.dump(policy_admin_money,policy_money_done)

### 태깅 합치기 from pickle only jiho


In [ ]:
# 대상연령시작, 대상연령 끝
age_done = pd.read_pickle('/content/drive/MyDrive/웰로/data/policy_age_done_2.pkl')
age_done = age_done[['정책ID', '대상연령_시작', '대상연령_끝']]

# 지역(시도), 지역(시군구)
loca_done = pd.read_pickle('/content/drive/MyDrive/웰로/data/policy_loca_done_2.pkl')
loca_done = loca_done[['정책ID', '필터링_시도', '필터링_시군구']]

# 대상특성, 대상특성상세
target_done = pd.read_pickle('/content/drive/MyDrive/웰로/data/policy_target_done2.pkl')
target_done = target_done[['정책ID', '필터링_대상특성', '필터링_대상특성상세']]

# 관심상황특성
situation_done = pd.read_pickle('/content/drive/MyDrive/웰로/data/policy_situation_done2.pkl')
situation_done = situation_done[['정책ID', '필터링_관심상황특성']]

# 중위소득
money_done = pd.read_pickle('/content/drive/MyDrive/웰로/data/policy_money_done.pkl')
money_done = money_done[['정책ID', '중위소득']]

# merge
my = policy.copy()
for plus in [age_done, loca_done, money_done, situation_done, target_done]:
  my = pd.merge(my, plus, how='inner', on='정책ID')

In [ ]:
my.columns

In [ ]:
my = my[['정책ID', '정책서비스ID', '서비스명', '소관기관', '소관기관유형.1' , '생애주기', '신청절차', '선정기준', '지원유형', '서비스목적', '지원내용' ,'지원대상', '대상연령_시작', '대상연령_끝', '필터링_시도', '필터링_시군구', '중위소득',
       '필터링_관심상황특성', '필터링_대상특성', '필터링_대상특성상세' ]]
my.rename(columns={'대상연령_시작':'필터링_대상연령시작', '대상연령_끝':'필터링_대상연령_끝', '중위소득':'필터링_중위소득'}, inplace=True)
my.columns

In [ ]:
my_done = open('/content/drive/MyDrive/웰로/data/jiho_done.pkl', 'wb')
pickle.dump(my,my_done)

### namjun start

In [58]:
policy.fillna('', inplace=True)

### 소관기관유형

#### 필터링 코드

In [59]:
def organization_authority(df):
  text = df['소관기관유형']
  if'중앙행정기관' in text:
    return '중앙행정기관'
  elif'공공기관' in text:
    return '공공기관 등'
  elif'교육청' in text:
    return '교육청'
  elif'광역시도' in text:
    return '광역시도'
  elif'시군구' in text:
    return '시군구'
  elif'민간기관' in text:
    return '민간기관'

org = []
for i in range(len(policy)):
  org.append(organization_authority(policy.iloc[i]))

In [60]:
policy_admin_org = pd.merge(left=policy, right=admin, how='inner', on='정책ID')
policy_admin_org = policy_admin_org[['정책ID', '소관기관유형']]

In [61]:
policy_admin_org['필터링_소관기관유형'] = org

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### 필터링 pickle dump

In [62]:
with open('/content/drive/MyDrive/웰로/data/policy_org.pkl', 'wb') as f:
    pickle.dump(policy_admin_org, f, pickle.HIGHEST_PROTOCOL)

### 지원유형 & 지원유형 상세

#### 필터링 코드

In [63]:
# 지원유형 & 지원유형상세 태그 추출
def support_type(df):
  
  a = df['지원유형'].replace('\n', '||').replace('\n\n', '||').split('||')
  tags, tag_d = [], []
  for i in a:
    if ('현금'in i) or ('융자' in i) or ('감면' in i) or ('대납' in i):                    
      tags.append('현금')
      if '장학금' in i:
        tag_d.append('현금(장학금)')     
      elif '융자' in i:
        tag_d.append('현금(융자)')
      elif ('감면' in i) or ('대납' in i):
        tag_d.append('현금(감면)')
      elif '보험' in i:
        tag_d.append('현금(보험)')
      else:                           #전체
        tag_d.append('현금(전체)')
    if ('현물' in i) or ('대여' in i ):
      tags.append('현물')
    if ('이용권'in i) or ('시설이용' in i):
      tags.append('이용권')
    if ('서비스' in i) and (i != '서비스이용권'):
      tags.append('서비스')
      if '주택임대' in i:           
        tag_d.append('서비스(주택임대)')
      elif '일자리' in i:
        tag_d.append('서비스(일자리)')
      elif '의료' in i:
        tag_d.append('서비스(의료)')
      elif '돌봄' in i:
        tag_d.append('서비스(돌봄)')
      else:                           #전체
        tag_d.append('서비스(전체)') 
    if '의료'in i:
      tags.append('의료지원')
    if (('상담'in i) or ('법률' in i)) and ('기타' not in i): 
      tags.append('상담/법률지원')
    if ('문화'in i) or ('여가' in i): 
      tags.append('문화/여가지원')
    if '민원'in i:
      tags.append('민원')
    if ('봉사'in i): 
      tags.append('봉사/기부')
    if ('기타'in i) or ('정보' in i):
      tags.append('기타')
      if ('교육' in i):
        tag_d.append('기타(교육)')
      elif ('상담' in i) or ('정보'):
        tag_d.append('기타(상담)')
      else:
        tag_d.append('기타(전체)')     
  tags = list(set(tags))
  if len(tags) == 0:
    tags = '기타'
  tag_d = list(set(tag_d))
  if len(tag_d) == 0:
    tag_d = '해당사항없음'
  return tags, tag_d

sup, sup_detailed = [], []
for i in range(len(policy)):
  sup.append(support_type(policy.iloc[i])[0])
  sup_detailed.append(support_type(policy.iloc[i])[1])

#### 지원유형 & 지원유형상세 맞춘 비율

In [64]:
policy_admin_support = pd.merge(left=policy, right=admin, how='inner', on='정책ID')
policy_admin_support = policy_admin_support[['정책ID', '지원유형_y','지원유형상세','지원대상']]

In [65]:
policy_admin_support['필터링_지원유형'] = sup
policy_admin_support['필터링_지원유형상세'] = sup_detailed

In [66]:
policy_admin_support['필터링_지원유형'] = policy_admin_support['필터링_지원유형'].map(lambda x: ','.join(x) if (type(x) == list) else (x if x == '해당사항없음' else x))
policy_admin_support['필터링_지원유형상세']= policy_admin_support['필터링_지원유형상세'].map(lambda x: ','.join(x) if (type(x) == list) else (x if x == '해당사항없음' else x))

In [67]:
#지원유형
print(len(policy_admin_support[policy_admin_support['지원유형_y'] == policy_admin_support['필터링_지원유형']]) / len(policy_admin_support) * 100)
#지원유형상세
print(len(policy_admin_support[policy_admin_support['지원유형상세'] == policy_admin_support['필터링_지원유형상세']]) / len(policy_admin_support) * 100)

14.173993626110345
0.0


#### 필터링 pickle dump

In [ ]:
with open('/content/drive/MyDrive/웰로/data/policy_support.pkl', 'wb') as f:
    pickle.dump(policy_admin_support, f, pickle.HIGHEST_PROTOCOL)

### 신청절차

#### 필터링 코드

In [68]:
# 신청절차 태그 추출

def application_methods(df):
  tokens = re.sub(r'[^가-힣]', ' ', df['신청절차'])  # 문장을 나눔
  tokens = tokens.split(' ')
  tok = []        
  for i in tokens:   
    if sum(i.count(x) for x in ['방문', '우편', '서면', '팩스', '은행', '영업접', '사무소', '내방']) >= 1 :
      tok.append('오프라인')
    elif sum(i.count(x) for x in ['모바일', 'ARS', '전화', '민원24', '온라인', '이메일', '인터넷', '홈택스', '홈페이지', '앱']) >= 1 :
      tok.append('온라인')
  if (('오프라인' in tok) and ('온라인' in tok)) or len(tok) == 0: # 아무 내용이 없을 경우, 복합으로 설정 > 추후 변경 필요시 이 부분 설정변경
    a = '무관'
  else:
    a = tok[0]
  return a

onoff = []
for i in range(len(policy)):
  onoff.append(application_methods(policy.iloc[i]))
 

#### 신청절차 맞춘 비율

In [69]:
policy_admin_app = pd.merge(left=policy, right=admin, how='inner', on='정책ID')
policy_admin_app = policy_admin_app[['정책ID', '신청절차_y']]

In [70]:
policy_admin_app['필터링_신청절차'] = onoff

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [71]:
#신청절차
print(len(policy_admin_app[policy_admin_app['신청절차_y'] == policy_admin_app['필터링_신청절차']]) / len(policy_admin_app) * 100)

32.56899396514703


#### 필터링 pickle dump

In [72]:
with open('/content/drive/MyDrive/웰로/data/policy_apply.pkl', 'wb') as f:
    pickle.dump(policy_admin_app, f, pickle.HIGHEST_PROTOCOL)

### 성별

#### 필터링 코드

In [73]:
# 성별 태그 추출
def gender(df):
  tok = []      
  tokens = re.sub(r'[^가-힣]', ' ', df['서비스명']+df['선정기준']+df['지원대상'])
  tokens = tokens.split(' ')
  for i in tokens:   # 토큰의 조건 별 태그 추출
    if (sum(i.count(x) for x in ['남성', '남자']) >= 2) or sum(i.count(x) for x in ['의경', '군인', '입대', '장병', '전립선']) >= 1 :
      tok.append('남성')
    elif (sum(i.count(x) for x in ['출산', '임산부', '임신']) >= 1) or (sum(i.count(x) for x in ['여성', '여자']) >= 2) :
      tok.append('여성')
  if len(set(tok)) == 1:  
    a = tok[0] 
  else:
    a = '무관'
  return a


gen = []
for i in range(len(policy)):
  gen.append(gender(policy.iloc[i]))

#### 성별 맞춘비율

In [74]:
policy_admin_gen = pd.merge(left=policy, right=admin, how='inner', on='정책ID')
policy_admin_gen = policy_admin_gen[['정책ID', '성별']]

In [75]:
policy_admin_gen['필터링_성별'] = gen

In [76]:
#성별
print(len(policy_admin_gen[policy_admin_gen['성별'] == policy_admin_gen['필터링_성별']]) / len(policy_admin_gen) * 100)

76.01654499016793


#### 필터링 pickle dump

In [ ]:
with open('/content/drive/MyDrive/웰로/data/policy_gender.pkl', 'wb') as f:
    pickle.dump(policy_admin_gen, f, pickle.HIGHEST_PROTOCOL)

### 결혼

#### 필터링 코드

In [77]:
#결혼 태그 추출
def marriage(df):
  
  tok = []
  tokens = df['서비스명'] + df['선정기준'] + df['지원대상']
  tokens = re.sub(r'[^가-힣]', ' ', tokens)
  tokens = tokens.split(' ')
  for i in tokens:
    if '미혼'in i:
      tok.append('미혼')
    if sum(i.count(x) for x in ['기혼', '가정', '아동', '다문화', '가족', '자녀']) >= 3 :
      tok.append('기혼')
    if '이혼' in i:
      tok.append('이혼')
    if '재혼' in i:
      tok.append('재혼')
  if len(tok)==0:
    a = '무관'
  else:
    a = list(set(tok))
  return a

marri = []
for i in range(len(policy)):
  marri.append(marriage(policy.iloc[i]))

#### 결혼 맞춘비율

In [78]:
policy_admin_marri = pd.merge(left=policy, right=admin, how='inner', on='정책ID')
policy_admin_marri = policy_admin_marri[['정책ID', '결혼']]

In [79]:
policy_admin_marri['필터링_결혼'] = marri

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
policy_admin_marri['필터링_결혼'] = policy_admin_marri['필터링_결혼'].map(lambda x: ','.join(x) if (type(x) == list) else (str(x) if ((type(x) == list) and (len(x) == 1)) else ( x if x == '무관' else x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [81]:
#결혼
print(len(policy_admin_marri[policy_admin_marri['결혼'] == policy_admin_marri['필터링_결혼']]) / len(policy_admin_marri) * 100)

63.84286779829578


#### 필터링 pickle dump

In [ ]:
with open('/content/drive/MyDrive/웰로/data/policy_marriage.pkl', 'wb') as f:
    pickle.dump(policy_admin_marri, f, pickle.HIGHEST_PROTOCOL)

### 학력

#### 필터링 코드

In [92]:

# 학력 태그 추출
def education(df):
  tokens= re.sub(r'[^가-힣]', ' ', df['서비스명'] + df['지원대상']+df['선정기준'])  # 문장을 모두 붙인다
  tokens = tokens.split(' ')
  tok= []
  tag = []
  for i in tokens: # i: 고등학교
    if sum(i.count(x) for x in ['고졸', '고등학교','고교생']) >= 1 :
      tok.append('고등학교졸업')
    if ('대학' in i) :
      tag.append(i)
    if ('휴학' in i): 
      tok.append('대학(원)휴학')
    if (('대학' in tag) & (('재학' in i)or ('학년' in i))) or ('대학생' in tag): 
      tok.append('대학(원)재학')
    if (('대학' in tag) & ('졸업' in i)) : 
      tok.append('대학(원)졸업')
  if len(tok)==0:
    a = '무관'
  else:
    a = list(set(tok))
  return a

edu = []
for i in range(len(policy)):
  edu.append(education(policy.iloc[i]))

#### 학력 맞춘비율

In [93]:
policy_admin_edu = pd.merge(left=policy, right=admin, how='inner', on='정책ID')
policy_admin_edu = policy_admin_edu[['정책ID', '학력']]

In [94]:
policy_admin_edu['필터링_학력'] = edu

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [95]:
policy_admin_edu['필터링_학력'] = policy_admin_edu['필터링_학력'].map(lambda x: ','.join(x) if (type(x) == list) else (str(x) if ((type(x) == list) and (len(x) == 1)) else ( x if x == '무관' else x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [96]:
# 학력
print(len(policy_admin_edu[policy_admin_edu['학력'] == policy_admin_edu['필터링_학력']]) / len(policy_admin_edu) * 100)

65.56291390728477


#### 필터링 pickle dump

In [ ]:
with open('/content/drive/MyDrive/웰로/data/policy_edu.pkl', 'wb') as f:
    pickle.dump(policy_admin_edu, f, pickle.HIGHEST_PROTOCOL)

### 가구원

#### 필터링 코드

In [87]:

# 가구원 태그 추출
def house_position(df):

  tokens_1 = re.sub(r'[^가-힣]', '', df['서비스명']+df['지원대상']+df['선정기준'])  # 문장을 모두 붙인다
  tok, tag, tag1, house = [], [], [], [] 
  if ('무주택세대주' in tokens_1) or ('무주택자' in tokens_1):
    tok.append('무주택 세대주')
  if ('무주택세대원' in tokens_1) or (('무주택자' in tokens_1 ) and ('세대원' in tokens_1)):
    tok.append('무주택 세대원')
  if ('주택소유' in  tokens_1)  or ('자가소유' in tokens_1) or ('자가를소유' in tokens_1) or ('주택을소유' in tokens_1):
    tok.append('주택 세대주')
    tag1.append('주택소유')
  tokens_2 = re.sub(r'[^가-힣]', ' ', df['지원대상']+df['선정기준'])  # 문장을 나눔
  tokens_2 = tokens_2.split(' ')

  for i in tokens_2:
    if ('무주택' in i): 
      tag.append(i)
    if (('임차인' in i ) or ('세대주' in i)) and ('무주택' in tag):
      tok.append('무주택 세대주')
    if (('세대원' in i) and ('무주택' in tag)) or (('구성원' in i) and ('무주택' in tag)):
      tok.append('무주택 세대원')
    if ('세대주' in i) and ('주택소유' in tag1): 
      tok.append('주택 세대주')
    if (('세대원' in i) and ('주택소유' in tag1)) or (('구성원' in i) and ('주택소유' in tag1)): 
      tok.append('주택 세대원')
  if len(tag) >= 1 and len(tok) == 0:
    tok.append('무주택 세대주')

  if len(tok) == 0:
    a = '무관'
  else:
    a = list(set(tok))
  return a

house = []
for i in range(len(policy)):
  house.append(house_position(policy.iloc[i]))


#### 가구원 맞춘비율

In [88]:
policy_admin_house = pd.merge(left=policy, right=admin, how='inner', on='정책ID')
policy_admin_house = policy_admin_house[['정책ID', '가구원']]

In [89]:
policy_admin_house['필터링_가구원'] = house

In [90]:
policy_admin_house['필터링_가구원'] = policy_admin_house['필터링_가구원'].map(lambda x: ','.join(x) if (type(x) == list) else (str(x) if ((type(x) == list) and (len(x) == 1)) else ( x if x == '무관' else x)))

In [91]:
# 가구원
print(len(policy_admin_house[policy_admin_house['가구원'] == policy_admin_house['필터링_가구원']]) / len(policy_admin_house) * 100)

81.5970888050087


#### 필터링 pickle dump

In [ ]:
with open('/content/drive/MyDrive/웰로/data/policy_house.pkl', 'wb') as f:
    pickle.dump(policy_admin_house, f, pickle.HIGHEST_PROTOCOL)

### 자녀 및 자녀상세

#### 필터링 코드

In [97]:
import re
# 자녀 및 자녀상세

def child(df):
  tok , age= [], []
  tokens = re.sub(r'[^가-힣]', ' ', df['서비스명']+df['지원대상']+df['선정기준'])
  tokens = tokens.split(' ')
  for i in tokens:
    if ('자녀' in i) or ('아동'in i): # 청소년의 경우 무관인 경우가 종종있는데, 
      tok.append('A07-01')       # 청소년도 함께 필터링 한 뒤, 리스트에 들어있으면 해당 안하도록 하는 방법 검토
  if len(tok) ==0:
    tok.append('A07-00')

  else:             
    for i in tokens:             # 자녀상세 시작
      if ('영아' in i) or ('유아' in i) or ('영유아' in i):
        tok.append('A07-01-00')
      if ('유치원' in i):
        tok.append('A07-01-01')
      if ('아동기' in i) or ('어린이' in i) or ('어린이집' in i) :
        tok.append('A07-01-02')
      if ('초중고' in i) or ('중고등' in i) or ('초등학생' in i) or ('중학생' in i) or ('고등학생' in i) or ('중학' in i):
        tok.append('A07-01-03')
      if ('고등학교' in i) or ('초등학교' in i) or ('중학교' in i):
        tok.append('태그')
      if ('재학' in i or '자녀' in i or '아동' in i or '학생') and (tok[:-2] == '태그'):
        tok.append('A07-01-03')
      if ('대학생' in i) or ('대학교'in i):
        tok.append('대학교')
      if ('재학' in i or '자녀' in i) and (tok[-1] == '대학교'):  
        tok.append('A07-01-04')
  
    ## 나이정보를 바탕으로 선별

    regex1 = '[0-9]{1,2} ?세 ?이상|[0-9]{1,2} ?세 ?이하|[0-9]{1,2} ?세 ?초과|[0-9]{1,2} ?세 ?미만'
    regex2 = '[0-9]{1,2} ?개월 ?이상|[0-9]{1,2} ?개월 ?이하|[0-9]{1,2} ?개월 ?초과|[0-9]{1,2} ?개월 ?미만'
    regex5 = '[0-9]{1,2} ?대 ?이상|[0-9]{1,2} ?대 ?이하|[0-9]{1,2} ?대 ?초과|[0-9]{1,2} ?대 ?미만'
    regex3 = '[0-9]{1,2}세? ?~ ?만? ?[0-9]{1,2}세' # 3~5세 | 6세~만19세
    regex4 = '[0-9]{1,2}개월? ?~ ?만? ?[0-9]{1,2}개월'   
    '''
    0~3세 영유아
    3~5세 유치원
    6~8세 어린이집
    8~12세 어린이    (초)
    13~18세 고등학생 (중고)
    18세~ 성인
    '''
    child_1 = range(0,3)  # 영유아
    child_2 = range(3,6)  # 유치원
    child_3 = range(6,9)  # 어린이집
    child_4 = range(9,19) # 초중고생
    child_5 = range(19, 200)  # 성인

    tag_dict = {child_1: 'A07-01-00',
    child_2: 'A07-01-01',
    child_3: 'A07-01-02',
    child_4: 'A07-01-03',
    child_5: 'A07-01-04'} 
    child_tags = [ child_1, child_2, child_3, child_4, child_5]
    r = re.compile(f'{regex1}|{regex2}|{regex3}|{regex4}|{regex5}')

    tokens = df['지원대상']+df['선정기준']
    tokens = r.findall(tokens)
    for i in tokens:

      if '~' in i :
        a = i.split('~')
        b = list(range(int(re.search('[0-9]+', a[0]).group()), int(re.search('[0-9]+', a[1]).group())+1))
        age.append(b)
      else:
        if '세' in i:
          k = re.findall('[0-9]+', i)
          if len(k) == 1:
            if ('이상' in i) or ('초과' in i):    # ex: 18세 이상 / 34세 초과
              age.append(list(range(int(k[0]), 20+1)))   
            else:                                 # ex: 18세 이하 / 18세 미만
              age.append(list(range(0, int(k[0])+1)))   
          elif len(k) == 2:   # ex: 8세 이상(초과) 18세 이하(미만)
            age.append(list(range(int(re.search('[0-9]+', a[0]).group()), int(re.search('[0-9]+', a[1]).group())+1)))  
          elif len(k) == 3:   # 3개는 출력으로 확인
            print(i)
        elif '개월' in i:
          k = re.findall('[0-9]+', i)
          if len(k) == 1:
            if ('이상' in i) or ('초과' in i):              # 18개월 이상 / 18개월 초과
              age.append(list(range(int(int(k[0])/12), 20+1)))
            else:                                           # 18개월 이하 / 18개월 미만
              age.append(list(range(0, int(int(k[0])/12)+1))) 
          elif len(k) == 2:                                      # 18개월 이상(초과) 36개월 이하(미만)
            age.append(list(range(int(re.search('[0-9]+', a[0]).group()), int(re.search('[0-9]+', a[1]).group())+1)))  
          elif len(k) == 3:   # 3개는 출력으로 확인
            print(i)
  
  if len(age) != 0:                                        # 나이 관련 정보가 있는경우
    for i in age: 
      if len(i) > 1:                                       # 나이가 1개 이상일 경우
        b =  min(i)                     
        c =  max(i)                     
        j = 0
        for ids in range(len(child_tags)):
          if b in child_tags[ids]:
            tok.append(tag_dict[child_tags[ids]])          # 첫번째 나이가 포함된 태그 추출
            j-=ids
          elif c in child_tags[ids]:
            while j >= 0:
              tok.append(tag_dict[child_tags[ids]])        # 두번째 나이가 포함된 태그 추출
              j -= 1
            j += 1
      else:                                                # 나이가 1개일 경우
        for ids in range(len(child_tags)):
          if i in child_tags[ids]:
            tok.append(tag_dict[child_tags[ids]])          # 해당 나이가 해당하는 zone의 태그를 추출
  a = list(set(tok))
  b = [ i for i in a if len(i) == 6]  # 자녀
  c = [ i for i in a if len(i) == 9]  # 자녀상세
  if len(c) == 0:
    c = '없음'
  tag_dict2 = {'A07-01-00': '영유아',
  'A07-01-01': '유치원',
  'A07-01-02': '어린이집',
  'A07-01-03': '초중고 학생',
  'A07-01-04': '성인',
  'A07-00' : '없음',
  'A07-01' : '있음'} 
  d = []
  for i in b:
    b.append(tag_dict2[i])
    b.remove(i)
  if '없음' in c:
    d.append('없음')
  else:
    for i in c:
      d.append(tag_dict2[i])
  return b, d


chi, chi_detailed = [], []
for i in range(len(policy)):
  chi.append(child(policy.iloc[i])[0])
  chi_detailed.append(child(policy.iloc[i])[1])


#### 자녀 및 자녀상세 맞춘비율

In [98]:
policy_admin_chi = pd.merge(left=policy, right=admin, how='inner', on='정책ID')
policy_admin_chi = policy_admin_chi[['정책ID', '자녀','자녀상세']]

In [99]:
policy_admin_chi['필터링_자녀'] = chi
policy_admin_chi['필터링_자녀상세'] = chi_detailed

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [100]:
policy_admin_chi.fillna('없음', inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [101]:
policy_admin_chi['필터링_자녀'] = policy_admin_chi['필터링_자녀'].map(lambda x: ','.join(x) if (type(x) == list) else (str(x) if ((type(x) == list) and (len(x) == 1)) else ( x if x == '해당사항없음' else x)))
policy_admin_chi['필터링_자녀상세'] = policy_admin_chi['필터링_자녀상세'].map(lambda x: ','.join(x) if (type(x) == list) else (str(x) if ((type(x) == list) and (len(x) == 1)) else ( x if x == '해당사항없음' else x)))

In [102]:
# 자녀
print(len(policy_admin_chi[policy_admin_chi['자녀'] == policy_admin_chi['필터링_자녀']]) / len(policy_admin_chi) * 100)
# 자녀 상세
print(len(policy_admin_chi[policy_admin_chi['자녀상세'] == policy_admin_chi['필터링_자녀상세']]) / len(policy_admin_chi) * 100)

80.21494925750966
82.0141039260448


많은 수의 중복 및 none값이 존재하여 다소 정확도가 떨어짐

---

#### 필터링 pickle dump

In [ ]:
with open('/content/drive/MyDrive/웰로/data/policy_child.pkl', 'wb') as f:
    pickle.dump(policy_admin_chi, f, pickle.HIGHEST_PROTOCOL)

### 직장 및 직장상세

#### 필터링 코드

In [103]:
policy_admin_work = pd.merge(policy, admin, how='inner', on='정책ID').fillna('')
policy_admin_work = policy_admin_work[['정책ID', '서비스명_x', '서비스목적', '지원대상','선정기준', '지원내용', '직장', '직장상세']]
policy_admin_work['text'] = policy_admin_work['서비스명_x'] + policy_admin_work['지원대상'] + policy_admin_work['선정기준'] + policy_admin_work['서비스목적'] + policy_admin_work['지원내용']

In [104]:
import re
work = []
work_detail = []

for i in range(len(policy_admin_work)):
  a = []
  b = []
  text = policy_admin_work.text.iloc[i]
  if '대학' in text:
    a.append('대학/대학원생')
    if '대학원' in text:
      b.append('대학생,대학원생')
    else:
      b.append('대학생')

  if sum(text.count(x) for x in ['구직', '실업', '일자리', '취업']) >= 2:
    a.append('구직자/실업자')
    if sum(text.count(x) for x in ['건설', '채굴']) >= 1:
      b.append('건설/채굴직')
    if sum(text.count(x) for x in ['경영',' 사무', '금융', '보험']) >= 1:
      b.append('경영/사무/금융/보험직')
    if sum(text.count(x) for x in ['교육', '법률', '사회복지', '경찰', '소방', '군인직', '공무원직']) >= 1:
      b.append('교육/법률/사회복지/경찰/소방/군인직/공무원직')
    if sum(text.count(x) for x in ['농림', '어업', '농업', '임업', '어업', '작물', '가축', '영림', '종축', '양잠']) >= 1:
      b.append('농림어업직')
    if sum(text.count(x) for x in ['미용', '여행', '숙박', '음식', '경비', '돌봄', '청소직']) >= 2:
      b.append('미용/여행/숙박/음식/경비/돌봄/청소직')
    if sum(text.count(x) for x in ['보건', '의료']) >= 1:
      b.append('보건/의료직')
    if sum(text.count(x) for x in ['설비', '정비', '생산직']) >= 1:
      b.append('설비/정비/생산직')
    if sum(text.count(x) for x in ['연구', '공학', '기술']) >= 2:
      b.append('연구/공학기술직')
    if sum(text.count(x) for x in ['영업', '판매', '운전' ,'운송']) >= 1:
      b.append('영업/판매/운전/운송직')
    if sum(text.count(x) for x in ['예술', '디자인', '방송', '스포츠']) >= 1:
      b.append('예술/디자인/방송/스포츠직')
  
  if (text.count('창업') >= 2) | (sum(text.count(x) for x in ['창업', '사업', '경영']) >= 3):
    a.append('예비창업가')
    if sum(text.count(x) for x in ['농업', '축산', '어업']) >= 2:
      b.append('농업/축산업/어업')
    if sum(text.count(x) for x in ['공공', '행정', '국방', '방위']) >= 2:
      b.append('공공/행정/국방/방위산업')
    if sum(text.count(x) for x in ['보건', '사회복지']) >= 1:
      b.append('보건업/사회복지사업')

  if sum(text.count(x) for x in ['근로자', '종사자','직원','직장']) >= 1 :
    a.append('근로자/직장인') 

  if text.count('소상공인') >= 2:
    a.append('소상공인')
  
  if ('중소기업' in text) | (sum(text.count(x) for x in ['사업', '기업', '법인', '경영'])  >= 4):
    a.append('중소기업')
  
  if '프리랜서' in text:
    a.append('프리랜서')
  
  if len(a) == 0:
    ans = '무관'
  else:
    ans = ','.join(a)
  
  if len(b) == 0:
    ans2 = None
  else:
    ans2 = ','.join(b)
  
  work.append(ans)
  work_detail.append(ans2)

In [105]:
# import re
# # 직장 및 직장상세

# def occupation(df):
#   tok, tok_detailed, univ, seeker, worker, pre_owner, s_owner, m_owner  = [], [], [], [], [], [], [], []
#   tokens = df['서비스명'] + df['지원대상'] + df['선정기준']
#   tokens = re.sub(r'[^가-힣]', ' ', tokens)
#   tokens = tokens.split(' ')

#   def occ_one(i, tok_detailed):
#     if ('건설업' in i) or ('채굴' in i):  
#       tok_detailed.append('건설/채굴직') 
#     if sum(i.count(x) for x in ['사무자동화','보험업','금융업','경영업','관리업','운영업','운영직']) >= 1 :
#       tok_detailed.append('경영/사무/금융/보험직') 
#     if sum(i.count(x) for x in ['교육','법률','사회복지','경찰','소방','군인직','공무원직']) >= 1 :
#       tok_detailed.append('교육/법률/사회복지/경찰/소방/군인직/공무원직')
#     if sum(i.count(x) for x in ['농림','축산','어업']) >= 1 :
#       tok_detailed.append('농림어업직')
#     if sum(i.count(x) for x in ['여행업','호텔업','숙박업','음식업','음시점','미용','음식숙박업','청소원','청소업','돌봄교실','노인복지','복지교육원']) >= 1 :
#       tok_detailed.append('미용/여행/숙박/음식/경비/돌봄/청소직')
#     if sum(i.count(x) for x in ['의사','간호사','약사','치료사','방사선사']) >= 1 :
#       tok_detailed.append('보건/의료직')
#     if sum(i.count(x) for x in ['설비업', '설비직','정비업', '정비직','생산', '생산업', '생산직']) >= 1 :
#       tok_detailed.append('설비/정비/생산직')
#     if sum(i.count(x) for x in ['환경산업','연구소','연구개발','연구자','기술직','과학자','과학기술']) >= 1 :
#       tok_detailed.append('연구/공학기술직')
#     if sum(i.count(x) for x in ['운전업','운전직','운송','운수','수송','화물','영업직','영업업무','판매원','판매직','판매업']) >= 1 :
#       tok_detailed.append('영업/판매/운전/운송직')
#     if sum(i.count(x) for x in ['예술','디자인','패션','방송','통신','체육회','선수','체육인','스포츠']) >= 1 :
#       tok_detailed.append('예술/디자인/방송/스포츠직')

#   def occ_two(i, tok_detailed):
#     if sum(i.count(x) for x in ['농업','농축','축산','어업']) >= 1 :
#       tok_detailed.append('농업/축산업/어업') 
#     if sum(i.count(x) for x in ['공공','비영리','행정','행정업무']) >= 1 :
#       tok_detailed.append('공공/행정/국방/방위산업')
#     if sum(i.count(x) for x in ['보건업','정신보건','보건사업','사회복지','복지사업']) >= 1 :
#       tok_detailed.append('보건업/사회복지사업')
#     if sum(i.count(x) for x in ['광업','임업']) >= 1 :
#       tok_detailed.append('광업/임업')
#     if ('교육사업' in i):
#       tok_detailed.append('교육 서비스업')
#     if ('서비스업' in i): # 요식업, 숙박업 등 기타 서비스업은 세분화 돼있으므로 서비스업 키워드만 진행
#       tok_detailed.append('서비스업')
#     if sum(i.count(x) for x in ['전문','과학','기술']) >= 1 :
#       tok_detailed.append('전문업/과학업/기술업') 
#     if ('제조' in i):
#       tok_detailed.append('제조업')
#     if sum(i.count(x) for x in ['협회','단체','조직','연합']) >= 1 :
#       tok_detailed.append('협회/단체/기타서비스업')
#     if sum(i.count(x) for x in ['정보제공','출판','방송']) >= 1 :
#       tok_detailed.append('출판업/방송업/정보서비스업')
#     if sum(i.count(x) for x in ['시설관리','택지개발','지역개발','도시개발']) >= 1 :
#       tok_detailed.append('사업시설관리업/지역개발사업')
#     if '건설업' in i:
#       tok_detailed.append('건설업')
#     if sum(i.count(x) for x in ['전기업','전기사업','가스사업','가스업','수도사업','하수도']) >= 1 :
#       tok_detailed.append('전기업/가스업/수도사업')
#     if sum(i.count(x) for x in ['예술', '여가','스포츠','체육시설','체육관','레저']) >= 1 :
#       tok_detailed.append('예술/스포츠/여가사업')
#     if sum(i.count(x) for x in ['도매','소매']) >= 1 :
#       tok_detailed.append('도매업/소매업')
#     if sum(i.count(x) for x in ['숙박','호텔','음식점','음식업','음식숙박업']) >= 1 :
#       tok_detailed.append('숙박업/음식점업')
#     if sum(i.count(x) for x in ['운수','운송','화물','여객']) >= 1 :
#       tok_detailed.append('운수업')
#     if sum(i.count(x) for x in ['부동산업','임대업','임대사업','건물주']) >= 1 :
#       tok_detailed.append('부동산업/임대업')
#     if sum(i.count(x) for x in ['금융업','보험업','보험사업','금융사업']) >= 1 :
#       tok_detailed.append('금융업/보험업')
#     if sum(i.count(x) for x in ['폐기물','환경산업']) >= 1 : #하수 폐기물이 없으므로 폐기물 처리시설로 치환해 진행
#       tok_detailed.append('하수폐기물처리업/환경복원업')

#   for i in tokens: 
#     if '대학' in i:
#       univ.append(i) # 대학/대학원생
#       tok.append('대학/대학원생')
#     if sum(i.count(x) for x in ['구직', '실업', '실직']) >= 1 :
#       seeker.append(i) # 구직자/실업자
#       tok.append('구직자/실업자')
#     if sum(i.count(x) for x in ['근로자', '종사자','직원','직장']) >= 1 :
#       worker.append(i) # 근로자/직장인
#       tok.append('근로자/직장인')
#     if '창업' in i:
#       pre_owner.append(i) # 예비창업자
#       tok.append('예비창업자')
#     if '소상' in i:         
#       s_owner.append(i) # 소상공인
#       tok.append('소상공인')
#     if ('중소' in i) or ('기업인' in i):       
#       m_owner.append(i) # 중소기업
#       tok.append('중소기업')
#     if ('프리랜서' in i):       
#       tok.append('프리랜서') # 프리랜서

#     # 대학생 / 대학원생 
#     if len(univ) != 0:
#       if ('대학원' in i) or (univ[-1] == '대학' and i == '원') :
#         tok_detailed.append('대학생')
#       elif ('대학생' in i) or (univ[-1] == '대학교' and '학생' in i) or (univ[-1] == '대학교' and '재학' in i): # 대학생 어떻게 거를지 고민해볼것
#         tok_detailed.append('대학원생')


#   if '무관' in tok or len(tok)==0:  # 1차적으로 태그추출 후 아무것도 없으면 무관 반환
#     return '무관', '해당사항없음'
#   else:
#     for i in tokens: 
      
#       # 구직자 / 실업자 / 실직자 / 취준생
#       if len(seeker) != 0:
#         occ_one(i, tok_detailed)
#       # 근로자
#       if len(worker) != 0:
#         occ_one(i, tok_detailed)
#       # 예비창업자
#       if len(pre_owner) != 0:
#         occ_two(i, tok_detailed)
#       # 소상공인
#       if len(s_owner) != 0:
#         occ_two(i, tok_detailed)
#       # 중소기업 
#       if len(m_owner) != 0:
#         occ_two(i, tok_detailed)

#     else:
#       b = list(set(tok)) # 직장
#       c = list(set(tok_detailed)) # 직장상세
#       if len(c) == 0:
#         c = '해당사항없음'
#       return b, c

# occ, occ_detailed = [], []
# for i in range(len(policy)):
#   occ.append(occupation(policy.iloc[i])[0])
#   occ_detailed.append(occupation(policy.iloc[i])[1])


#### 직장 및 직장상세 맞춘비율

In [106]:
policy_admin_occ = pd.merge(left=policy, right=admin, how='inner', on='정책ID')
policy_admin_occ = policy_admin_occ[['정책ID', '직장','직장상세']]

In [107]:
# policy_admin_occ['필터링_직장'] = occ
# policy_admin_occ['필터링_직장상세'] = occ_detailed

In [108]:
# policy_admin_occ['필터링_직장'] = policy_admin_occ['필터링_직장'].map(lambda x: ','.join(x) if (type(x) == list) else (str(x) if ((type(x) == list) and (len(x) == 1)) else ( x if x == '해당사항없음' else x)))
# policy_admin_occ['필터링_직장상세'] = policy_admin_occ['필터링_직장상세'].map(lambda x: ','.join(x) if (type(x) == list) else (str(x) if ((type(x) == list) and (len(x) == 1)) else ( x if x == '해당사항없음' else x)))

In [109]:
policy_admin_occ.직장.fillna('무관', inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [110]:
# 죠
policy_admin_occ['필터링_직장'] = work
policy_admin_occ['필터링_직장상세'] = work_detail

# 직장
print(len(policy_admin_occ[policy_admin_occ['직장'] == policy_admin_occ['필터링_직장']]) / len(policy_admin_occ) * 100)

# 직장상세
print(len(policy_admin_occ[policy_admin_occ['직장상세'] == policy_admin_occ['필터링_직장상세']]) / len(policy_admin_occ) * 100)

53.61187080442104
0.4023235314060981


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [111]:
policy_admin_occ['직장'].value_counts()

무관                                              59621
구직자/실업자, 근로자/직장인                                 4901
소상공인, 중소기업                                       4779
근로자/직장인                                          4560
중소기업                                             1987
                                                ...  
소상공인, 근로자/직장인, 구직자/실업자, 중소기업, 예비창업자, 프리랜서           1
대학/대학원생, 구직자/실업자, 근로자/직장인, 예비창업자, 소상공인, 프리랜서        1
대학/대학원생, 소상공인, 중소기업                                 1
대학/대학원생, 예비창업자, 중소기업                                1
대학/대학원생, 구직자/실업자, 예비창업자, 소상공인, 중소기업                 1
Name: 직장, Length: 96, dtype: int64

#### 필터링 pickle dump

In [ ]:
with open('/content/drive/MyDrive/웰로/data/policy_occ.pkl', 'wb') as f:
    pickle.dump(policy_admin_occ, f, pickle.HIGHEST_PROTOCOL)

### 태깅 합치기 from pickle namjoons' mixed

In [ ]:
# 소관기관유형
org_done = pd.read_pickle('/content/drive/MyDrive/웰로/data/policy_org.pkl')
org_done = org_done[['정책ID', '필터링_소관기관유형']]

# 지원유형, 지원유형상세
sup_done = pd.read_pickle('/content/drive/MyDrive/웰로/data/policy_support.pkl')
sup_done = sup_done[['정책ID', '필터링_지원유형', '필터링_지원유형상세']]

# 신청절차
app_done = pd.read_pickle('/content/drive/MyDrive/웰로/data/policy_apply.pkl')
app_done = app_done[['정책ID', '필터링_신청절차']]

# 성별
gender_done = pd.read_pickle('/content/drive/MyDrive/웰로/data/policy_gender.pkl')
gender_done = gender_done[['정책ID', '필터링_성별']]

# 결혼
marri_done = pd.read_pickle('/content/drive/MyDrive/웰로/data/policy_marriage.pkl')
marri_done = marri_done[['정책ID', '필터링_결혼']]

# 학력
edu_done = pd.read_pickle('/content/drive/MyDrive/웰로/data/policy_edu.pkl')
edu_done = edu_done[['정책ID', '필터링_학력']]

# 가구원
house_done = pd.read_pickle('/content/drive/MyDrive/웰로/data/policy_house.pkl')
house_done = house_done[['정책ID', '필터링_가구원']]

# 자녀, 자녀상세
child_done = pd.read_pickle('/content/drive/MyDrive/웰로/data/policy_child.pkl')
child_done = child_done[['정책ID', '필터링_자녀','필터링_자녀상세']]

# 직장, 직장상세
occ_done = pd.read_pickle('/content/drive/MyDrive/웰로/data/policy_occ.pkl')
occ_done = occ_done[['정책ID', '필터링_직장','필터링_직장상세']]




# merge
nam = policy.copy()['정책ID']
for plus in [org_done, sup_done, app_done, gender_done, marri_done, edu_done, house_done, child_done, occ_done]:
  nam = pd.merge(nam, plus, how='inner', on='정책ID')

In [ ]:
nam.columns

In [ ]:
with open('/content/drive/MyDrive/웰로/data/nam_done0117.pkl', 'wb') as f:
    pickle.dump(nam, f, pickle.HIGHEST_PROTOCOL)

### MB15(관심정책)



#### mb15 필터링 코드

In [ ]:

with open('/content/drive/MyDrive/웰로/data/jiho_done.pkl', 'rb') as f:
    jiho = pickle.load(f)

In [ ]:
# jiho.rename(columns = {'신청절차' : '정책신청절차'}, inplace=True)
jiho.rename(columns = {'지원유형' : '정책지원유형'}, inplace=True)

In [ ]:
jiho.columns

In [ ]:
policy_admin_support.columns

In [ ]:
policy_mb15 = policy.copy()
policy_mb15 = policy_mb15.fillna('')
A,B,C,D = [], [], [], []

for i, j, k  in zip(jiho['필터링_관심상황특성'], jiho['필터링_대상연령시작'], jiho['필터링_대상연령_끝']):
  a= []
  if ('개인교육지원' in i) and ((j >= 0) and (k < 8 )):               # 개인 교육 나이 별로 나눠야 함
    a.append('보육지원(0~7세)')
  if ('개인교육지원' in i) and ((j >= 8) and (k < 19)):  
    a.append('교육지원(8~19세)')
  if ('개인교육지원' in i) and (j >= 19):    
    a.append('성인교육지원')
  if ('개인금융지원' in i):
    a.append('개인금융지원')
  if ('기업금융지원' in i):
    a.append('기업금융지원')
  A.append(a)
for i, j, k, l, m, n in zip(policy_admin_support['필터링_지원유형상세'], jiho['필터링_대상연령시작'], jiho['필터링_대상연령_끝'], policy_admin_support['지원대상'], policy_admin_support['필터링_지원유형'], jiho['서비스명']):
  b = []
  if ('현금' in i) or (sum(n.count(x) for x in ['경비', '자조금', '구매','금융', '료', '직불']) >= 1):
    if sum(l.count(x) for x in ['기업', '법인', '단체', '선사', '공장', '운영사', '시설', '경영체', '센터' , '업체']) >= 1:
      b.append('기업금융지원')
    if sum(l.count(x) for x in ['개인', '아동', '업인', '업자', '가구', '생산자', '거주자', '가족', '경영인', '주민', '본인', '배우자', '사람', '친족', '피해자', '신고자', '군인', '유족']) >= 1:
      b.append('개인금융지원')
    else:
      b.append('개인금융지원')    
  if (('교육' in i) and ((j >= 0) and (k < 8 ))) or ('돌봄' in i) or ('영유아' in n )or ('유아' in n ) or('어린이' in n):   # 기타(교육) 또는 서비스(돌봄)         
    b.append('보육지원(0~7세)')
  if (('교육' in i) and ((j >= 8) and (k < 19))) or ('청소년' in n):  
    b.append('교육지원(8~19세)')
  if ('교육' in i) and (j >= 19):   
    b.append('성인교육지원')
  if ('문화' in m ):          
    b.append('문화생활 지원')
  if ('의료' in m) or ('피해' in n) or ('심리' in n) or ('치료' in n) or ('인권' in n) or ('상담' in n):
    b.append('의료 지원')
  if ('주택' in i) or ('주택' in l) or ('부동산' in l) :
    b.append('주택-부동산 지원')
  if ('일자리' in i) or ('준비생' in l ) or ('구직' in l) or ('취업' in l):
    b.append('취업지원')
  B.append(b)
for i in policy_admin_occ['필터링_직장']:
  c= [] 
  if ('대학'in i):
    c.append('취업 지원')
  if ('창업' in i) or ('창업' in n):
    c.append('창업 지원')
  if ('근로자' in i) or (sum(n.count(x) for x in ['관리','운영']) >= 1):
    c.append('근로자 지원')
  C.append(c)




jiho['A'] = A
jiho['B'] = B
jiho['C'] = C
jiho['MB_15'] = jiho['A'] +jiho['B'] +jiho['C']
jiho.drop(columns= ['A','B','C'], inplace = True)

In [ ]:
policy_admin_occ['필터링_직장'].value_counts()

In [ ]:
jiho['MB_15'] = jiho['MB_15'].map(lambda x: ','.join(x) if type(x) == list else x)

In [ ]:
ourfinal = pd.merge(jiho, nam, how='inner', on='정책ID')

In [ ]:
ourfinal.rename(columns = {'신청절차': '정책신청절차'}, inplace=True)

In [ ]:
rename_dict= {}
for name in ourfinal.columns:
  if '필터링' in name:
    rename_dict[name] = name[4:]
rename_dict

ourfinal.rename(columns = rename_dict, inplace=True)

In [ ]:
ourfinal.columns

In [ ]:
ourfinal.rename(columns = {'대상연령_끝': '대상연령끝'}, inplace=True)

In [ ]:
with open('/content/drive/MyDrive/웰로/data/our_final0117.pkl', 'wb') as f:
  pickle.dump(ourfinal, f, pickle.HIGHEST_PROTOCOL)

# 추천 모델 (유사도)

In [ ]:

######### our_final0117 이 필요하다!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
policy = pd.DataFrame(pd.read_pickle('/content/drive/MyDrive/웰로/data/our_final0117.pkl'))
user = pd.DataFrame(pd.read_pickle('/content/drive/MyDrive/웰로/data/user_final0117.pkl'))

In [ ]:
policy.columns

Index(['정책ID', '정책서비스ID', '서비스명', '소관기관', '소관기관유형.1', '생애주기', '정책신청절차', '선정기준',
       '정책지원유형', '서비스목적', '지원내용', '지원대상', '대상연령시작', '대상연령끝', '시도', '시군구',
       '중위소득', '관심상황특성', '대상특성', '대상특성상세', 'MB_15', '소관기관유형', '지원유형', '지원유형상세',
       '신청절차', '성별', '결혼', '학력', '가구원', '자녀', '자녀상세', '직장', '직장상세'],
      dtype='object')

In [ ]:
user.columns

Index(['num', '성별', '나이', '시도', '시군구', '학력', '직장', '가구원', '결혼', '자녀', '자녀상세',
       '자녀수', 'mb_10', 'mb_10+대상특성', 'mb_11', 'mb_11+관심상황특성', 'mb_12', 'mb_13',
       'mb_14', 'mb_15'],
      dtype='object')

In [ ]:
policy.head(1)

,정책ID,정책서비스ID,서비스명,소관기관,소관기관유형.1,생애주기,정책신청절차,선정기준,정책지원유형,서비스목적,지원내용,지원대상,대상연령시작,대상연령끝,시도,시군구,중위소득,관심상황특성,대상특성,대상특성상세,MB_15,소관기관유형,지원유형,지원유형상세,신청절차,성별,결혼,학력,가구원,자녀,자녀상세,직장,직장상세
0,1,000000465790,만 3~5세 유치원 유아 학비 지원(3~5세 누리과정 운영),교육부,중앙정부,영·유아||아동·청소년||청년||중장년||노년기,신청ㆍ접수(읍면동사무소) - 보장 결정(시군구 사업팀),1) 지원대상 : 어린이집을 이용하는 만 3~5세 아동\n2) 신청인 : 아동의 보...,현금,누리과정이란 만 3~5세를 대상으로 하는 어린이집 표준보육과정과 유치원 교육과정을 ...,"만 3~5세 누리과정 운영에 필요한 비용 지원 \n - 보육료, 담당교사 처우 개선...",지원대상 : 어린이집을 이용하는 만 3~5세 아동\n- 취학유예 : 만 6세아 초등...,3,5,전국,None,None,개인교육지원,해당사항없음,,"보육지원(0~7세),개인금융지원,보육지원(0~7세)",중앙행정기관,현금,현금(전체),오프라인,무관,무관,무관,무관,있음,"어린이집,유치원,영유아",무관,None


In [ ]:
user.head(1)

,num,성별,나이,시도,시군구,학력,직장,가구원,결혼,자녀,자녀상세,자녀수,mb_10,mb_10+대상특성,mb_11,mb_11+관심상황특성,mb_12,mb_13,mb_14,mb_15
0,11726,남성,121,충청북도,청주시,무관,무관,무관,"미혼,기혼","있음,없음",해당사항없음,None,None,해당사항없음,None,None,None,None,None,None


## 유저 * feature matrix

### 유저 * feature matrix 12개 (밑 11개 + age) for 보편

In [ ]:
def age_matrix(value):
  user_age = np.array([ [ 0 for i in range(len(policy))] for j in range(1)])
  if value > 99:
    value = 99
  for start, end in zip(enumerate(policy.대상연령시작), enumerate(policy.대상연령끝)):
    i, start = start
    i, end = end
    if start <= value <= end:
      user_age[0][i] = 1
  return user_age

In [ ]:
def gender_matrix(value):
  f = 2
  u = 1
  user_gender = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if '남성' in value:
    user_gender[0][0] = 1
  if '여성' in value:
    user_gender[0][1] = 1
  return user_gender

def city_matrix(value):
  city = ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도' ]
  f = len(city)
  u = 1
  user_city = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if '전국' in value: #전국
    for i in range(f):
      user_city[0][i] = 1
    return user_city
  
  for j, city_name in enumerate(city):
    if city_name in value:
      user_city[0][j] = 1
  return user_city

def city2_matrix(value):
  city2 = ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', '중구', '서구', '동구', '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구', '금정구', '강서구', '연제구', '수영구', '사상구', '기장군', '중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군', '중구', '동구', '미추홀구', '연수구', '남동구', '부평구', '계양구', '서구', '강화군', '옹진군', '동구', '서구', '남구', '북구', '광산구', '동구', '중구', '서구', '유성구', '대덕구', '중구', '남구', '동구', '북구', '울주군', '세종시', '수원시', '성남시', '고양시', '용인시', '부천시', '안산시', '안양시', '남양주시', '화성시', '평택시', '의정부시', '시흥시', '파주시', '광명시', '김포시', '군포시', '광주시', '이천시', '양주시', '오산시', '구리시', '안성시', '포천시', '의왕시', '하남시', '여주시', '양평군', '동두천시', '과천시', '가평군', '연천군', '춘천시', '원주시', '강릉시', '동해시', '태백시', '속초시', '삼척시', '홍천군', '횡성군', '영월군', '평창군', '정선군', '철원군', '화천군', '양구군', '인제군', '고성군', '양양군', '청주시', '충주시', '제천시', '보은군', '옥천군', '영동군', '진천군', '괴산군', '음성군', '단양군', '증평군', '천안시', '공주시', '보령시', '아산시', '서산시', '논산시', '계룡시', '당진시', '금산군', '부여군', '서천군', '청양군', '홍성군', '예산군', '태안군', '전주시', '군산시', '익산시', '정읍시', '남원시', '김제시', '완주군', '진안군', '무주군', '장수군', '임실군', '순창군', '고창군', '부안군', '목포시', '여수시', '순천시', '나주시', '광양시', '담양군', '곡성군', '구례군', '고흥군', '보성군', '화순군', '장흥군', '강진군', '해남군', '영암군', '무안군', '함평군', '영광군', '장성군', '완도군', '진도군', '신안군', '포항시', '경주시', '김천시', '안동시', '구미시', '영주시', '영천시', '상주시', '문경시', '경산시', '군위군', '의성군', '청송군', '영양군', '영덕군', '청도군', '고령군', '성주군', '칠곡군', '예천군', '봉화군', '울진군', '울릉군', '창원시', '진주시', '통영시', '사천시', '김해시', '밀양시', '거제시', '양산시', '의령군', '함안군', '창녕군', '고성군', '남해군', '하동군', '산청군', '함양군', '거창군', '합천군', '제주시', '서귀포시']
  f = len(city2)
  u = 1
  user_city2 = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if '' == value:
    # for i in range(f):
    #   user_city2[0][i] = 0
    return user_city2

  for j, city_name in enumerate(city2):
    if city_name in value:
      user_city2[0][j] = 1
  return user_city2

def edu_matrix(value):
  f = 4
  u = 1
  user_edu = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if '무관' in value:
    for i in range(f):
      user_edu[0][i] = 1
    return user_edu
  if '고등학교졸업' in value:
    user_edu[0][0] = 1
  elif '대학(원)재학' in value:
    user_edu[0][1] = 1
  elif '대학(원)휴학' in value:
    user_edu[0][2] = 1
  elif '대학(원)졸업' in value:
    user_edu[0][3] = 1
  return user_edu

def work_matrix(value):
  f = 7
  u = 1
  user_work = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if '무관' in value:
    for i in range(f):
      user_work[0][i] = 1
    return user_work
  if '대학/대학원생' in value:
    user_work[0][0] = 1
  if '구직자/실업자' in value:
    user_work[0][1] = 1
  if '근로자/직장인' in value:
    user_work[0][2] = 1
  if '예비창업자' in value:
    user_work[0][3] = 1
  if '소상공인' in value:
    user_work[0][4] = 1
  if '중소기업' in value:
    user_work[0][5] = 1
  if '프리랜서' in value:
    user_work[0][6] = 1
  return user_work

def house_matrix(value):
  f = 4
  u = 1
  user_house = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if '무관' in value:
    for i in range(4):
      user_house[0][i] = 1
    return user_house
  if '무주택 세대주' in value:
    user_house[0][0] = 1
  if '무주택 세대원' in value:
    user_house[0][1] = 1
  if '주택 세대주' in value:
    user_house[0][2] = 1
  if '주택 세대원' in value:
    user_house[0][3] = 1
  return user_house

def marry_matrix(value):
  f = 2
  u = 1
  user_marry = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if '미혼' in value: #미혼
    user_marry[0][0] = 1
  if '기혼' in value: #기혼
    user_marry[0][1] = 1
  return user_marry

def child_matrix(value):
  f = 2
  u = 1
  user_child = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if (value== None) or (value=='있음,없음'):
    user_child[0][0] = 1
    user_child[0][1] = 1
  elif value== '없음': # 없음
    user_child[0][0] = 1
  elif value== '있음': # A-07-01
    user_child[0][1] = 1
  return user_child

def child_detail_matrix(value):
  f = 5
  u = 1
  user_child_detail = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if (value== None) or (value == '해당사항없음'):
    # for i in range(f):
    #   user_child_detail[0][i] = 0
    return user_child_detail
  if '영유아' in value: # 영유아
    user_child_detail[0][0] = 1
  if '유치원' in value:
    user_child_detail[0][1] = 1
  if '어린이집' in value:
    user_child_detail[0][2] = 1
  if '초중고학생' in value:
    user_child_detail[0][3] = 1
  if '성인' in value:
    user_child_detail[0][4] = 1
  return user_child_detail

def mb_15_matrix(value):
  mb_15 = ['근로자 지원', '취업 지원', '개인금융지원', '문화생활 지원', '주택-부동산 지원', '교육지원(만8~19세)', '성인교육지원',
         '의료 지원', '창업 지원', '기업금융지원', '보육지원(만0~7세)']
  f = len(mb_15)
  u = 1
  user_mb_15 = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if ('' == value) or (value == None):
    for i in range(f):
      user_mb_15[0][i] = 1
    return user_mb_15
  for i, text in enumerate(mb_15):
    if text in value:
      word_0 = mb_15.index(text)
      user_mb_15[0][word_0] = 1
  return user_mb_15

def money_matrix(value):
  mb_12 = ['기초생활수급자', '중위소득 40% 이하', '중위소득 40~60% 사이', '중위소득 60~80% 사이', '중위소득 80~100% 사이',  '중위소득 100% 이상']
  f = len(mb_12)
  u = 1
  if value == None:
    for i in range(f):
      user_money = np.array([[1 for i in range(f)] for j in range(u)])
    return user_money
  
  user_money = np.array([ [ 0 for i in range(f)] for j in range(u)])
  
  if '' == value:
    for i in range(f):
      user_money[0][i] = 1
    return user_money
  for i, text in enumerate(mb_12):
    if text in value:
      word_0 = mb_12.index(text)
      user_money[0][word_0] = 1
  return user_money

### 유저 * feature matrix 12개 (밑 11개 + age) for 특수

In [ ]:
def age_matrix2(value):
  user_age = np.array([ [ 0 for i in range(len(policy))] for j in range(1)])
  if value > 99:
    value = 99
  for start, end in zip(enumerate(policy.대상연령시작), enumerate(policy.대상연령끝)):
    i, start = start
    i, end = end
    if start <= value <= end:
      user_age[0][i] = 1
  return user_age

In [ ]:
def gender_matrix2(value):
  f = 2
  u = 1
  user_gender = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if '남성' in value:
    user_gender[0][0] = 1
  if '여성' in value:
    user_gender[0][1] = 1
  return user_gender

def city_matrix2(value):
  city = ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도' ]
  f = len(city)
  u = 1
  user_city = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if '전국' in value: #전국
    for i in range(f):
      user_city[0][i] = 1
    return user_city
  
  for j, city_name in enumerate(city):
    if city_name in value:
      user_city[0][j] = 1
  return user_city

def city2_matrix2(value):
  city2 = ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', '중구', '서구', '동구', '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구', '금정구', '강서구', '연제구', '수영구', '사상구', '기장군', '중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군', '중구', '동구', '미추홀구', '연수구', '남동구', '부평구', '계양구', '서구', '강화군', '옹진군', '동구', '서구', '남구', '북구', '광산구', '동구', '중구', '서구', '유성구', '대덕구', '중구', '남구', '동구', '북구', '울주군', '세종시', '수원시', '성남시', '고양시', '용인시', '부천시', '안산시', '안양시', '남양주시', '화성시', '평택시', '의정부시', '시흥시', '파주시', '광명시', '김포시', '군포시', '광주시', '이천시', '양주시', '오산시', '구리시', '안성시', '포천시', '의왕시', '하남시', '여주시', '양평군', '동두천시', '과천시', '가평군', '연천군', '춘천시', '원주시', '강릉시', '동해시', '태백시', '속초시', '삼척시', '홍천군', '횡성군', '영월군', '평창군', '정선군', '철원군', '화천군', '양구군', '인제군', '고성군', '양양군', '청주시', '충주시', '제천시', '보은군', '옥천군', '영동군', '진천군', '괴산군', '음성군', '단양군', '증평군', '천안시', '공주시', '보령시', '아산시', '서산시', '논산시', '계룡시', '당진시', '금산군', '부여군', '서천군', '청양군', '홍성군', '예산군', '태안군', '전주시', '군산시', '익산시', '정읍시', '남원시', '김제시', '완주군', '진안군', '무주군', '장수군', '임실군', '순창군', '고창군', '부안군', '목포시', '여수시', '순천시', '나주시', '광양시', '담양군', '곡성군', '구례군', '고흥군', '보성군', '화순군', '장흥군', '강진군', '해남군', '영암군', '무안군', '함평군', '영광군', '장성군', '완도군', '진도군', '신안군', '포항시', '경주시', '김천시', '안동시', '구미시', '영주시', '영천시', '상주시', '문경시', '경산시', '군위군', '의성군', '청송군', '영양군', '영덕군', '청도군', '고령군', '성주군', '칠곡군', '예천군', '봉화군', '울진군', '울릉군', '창원시', '진주시', '통영시', '사천시', '김해시', '밀양시', '거제시', '양산시', '의령군', '함안군', '창녕군', '고성군', '남해군', '하동군', '산청군', '함양군', '거창군', '합천군', '제주시', '서귀포시']
  f = len(city2)
  u = 1
  user_city2 = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if '' == value:
    # for i in range(f):
    #   user_city2[0][i] = 0
    return user_city2

  for j, city_name in enumerate(city2):
    if city_name in value:
      user_city2[0][j] = 1
  return user_city2

def edu_matrix2(value):
  f = 4
  u = 1
  user_edu = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if '무관' in value:
    # for i in range(f):
    #   user_edu[0][i] = 1
    return user_edu
  if '고등학교졸업' in value:
    user_edu[0][0] = 1
  elif '대학(원)재학' in value:
    user_edu[0][1] = 1
  elif '대학(원)휴학' in value:
    user_edu[0][2] = 1
  elif '대학(원)졸업' in value:
    user_edu[0][3] = 1
  return user_edu

def work_matrix2(value):
  f = 7
  u = 1
  user_work = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if '무관' in value:
    # for i in range(f):
    #   user_work[0][i] = 1
    return user_work
  if '대학/대학원생' in value:
    user_work[0][0] = 1
  if '구직자/실업자' in value:
    user_work[0][1] = 1
  if '근로자/직장인' in value:
    user_work[0][2] = 1
  if '예비창업자' in value:
    user_work[0][3] = 1
  if '소상공인' in value:
    user_work[0][4] = 1
  if '중소기업' in value:
    user_work[0][5] = 1
  if '프리랜서' in value:
    user_work[0][6] = 1
  return user_work

def house_matrix2(value):
  f = 4
  u = 1
  user_house = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if '무관' in value:
    # for i in range(4):
    #   user_house[0][i] = 1
    return user_house
  if '무주택 세대주' in value:
    user_house[0][0] = 1
  if '무주택 세대원' in value:
    user_house[0][1] = 1
  if '주택 세대주' in value:
    user_house[0][2] = 1
  if '주택 세대원' in value:
    user_house[0][3] = 1
  return user_house

def marry_matrix2(value):
  f = 2
  u = 1
  user_marry = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if '미혼' in value: #미혼
    user_marry[0][0] = 1
  if '기혼' in value: #기혼
    user_marry[0][1] = 1
  return user_marry

def child_matrix2(value):
  f = 2
  u = 1
  user_child = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if (value== None):
    return user_child
  elif (value=='있음,없음'):
    user_child[0][0] = 1
    user_child[0][1] = 1
  elif value== '없음': # 없음
    user_child[0][0] = 1
  elif value== '있음': # A-07-01
    user_child[0][1] = 1
  return user_child

def child_detail_matrix2(value):
  f = 5
  u = 1
  user_child_detail = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if (value== None) or (value == '해당사항없음'):
    # for i in range(f):
    #   user_child_detail[0][i] = 0
    return user_child_detail
  if '영유아' in value: # 영유아
    user_child_detail[0][0] = 1
  if '유치원' in value:
    user_child_detail[0][1] = 1
  if '어린이집' in value:
    user_child_detail[0][2] = 1
  if '초중고학생' in value:
    user_child_detail[0][3] = 1
  if '성인' in value:
    user_child_detail[0][4] = 1
  return user_child_detail

def mb_15_matrix2(value):
  mb_15 = ['근로자 지원', '취업 지원', '개인금융지원', '문화생활 지원', '주택-부동산 지원', '교육지원(만8~19세)', '성인교육지원',
         '의료 지원', '창업 지원', '기업금융지원', '보육지원(만0~7세)']
  f = len(mb_15)
  u = 1
  user_mb_15 = np.array([ [ 0 for i in range(f)] for j in range(u)])
  if ('' == value) or (value == None):
    # for i in range(f):
    #   user_mb_15[0][i] = 0
    return user_mb_15
  for i, text in enumerate(mb_15):
    if text in value:
      word_0 = mb_15.index(text)
      user_mb_15[0][word_0] = 1
  return user_mb_15

def money_matrix2(value):
  mb_12 = ['기초생활수급자', '중위소득 40% 이하', '중위소득 40~60%', '중위소득 60~80%', '중위소득 80~100%',  '중위소득 100% 이상']
  f = len(mb_12)
  u = 1
  user_money = np.array([[0 for i in range(f)] for j in range(u)])
  if (value == None) or ('' == value):
    # for i in range(f):
    #   user_money = np.array([[1 for i in range(f)] for j in range(u)])
    return user_money

  for i, text in enumerate(mb_12):
    if text in value:
      word_0 = mb_12.index(text)
      user_money[0][word_0] = 1
  return user_money

## feature * policy matrix

### feature * policy matrix 11개 for 보편

#### child

In [ ]:

u = len(user)
f = 2
p = len(policy)

In [ ]:
child_policy = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
child_policy.shape

(88486, 2)

In [ ]:
for idx, value in enumerate(policy.자녀.to_numpy()):
  if value== '없음': # 없음
    child_policy[idx][0] = 1
  if value== '있음': # 있음
    child_policy[idx][1] = 1

In [ ]:
# child_policy_df = pd.DataFrame(child_policy, columns=['없음', '있음'])
child_policy = child_policy.T

#### child_detail

In [ ]:
u = len(user)
f = 5
p = len(policy)

In [ ]:
child_detail_policy = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
child_detail_policy.shape

(88486, 5)

In [ ]:
for idx, value in enumerate(policy.자녀상세.to_numpy()):
  if (value== '해당사항없음'):
    # for i in range(5):
    #   child_detail_policy[idx][i] = 0
    continue
  if '영유아' in value:
    child_detail_policy[idx][0] = 1
  if '유치원' in value:
    child_detail_policy[idx][1] = 1
  if '어린이집' in value:
    child_detail_policy[idx][2] = 1
  if '초중고학생' in value:
    child_detail_policy[idx][3] = 1
  if '성인' in value:
    child_detail_policy[idx][4] = 1

In [ ]:
# child_detail_policy_df = pd.DataFrame(child_detail_policy, columns = ['영유아', '유치원', '어린이집', '초중고학생', '성인'])
child_detail_policy = child_detail_policy.T

#### house

In [ ]:
u = len(user)
f = 4
p = len(policy)

In [ ]:
house_policy = np.array([ [ 0 for i in range(f) ] for j in range(p)])
house_policy.shape

(88486, 4)

In [ ]:
for idx, value in enumerate(policy.가구원.to_numpy()):
  if '무관' in value:
    for i in range(4):
      house_policy[idx][i] = 1
    continue
  if '무주택 세대주' in value:
    house_policy[idx][0] = 1
  if '무주택 세대원' in value:
    house_policy[idx][1] = 1
  if '주택 세대주' in value:
    house_policy[idx][2] = 1
  if '주택 세대원' in value:
    house_policy[idx][3] = 1

In [ ]:
# house_policy_df = pd.DataFrame(house_policy, columns = ['무관', '무주택 세대주', '무주택 세대원', '주택 세대주', '주택 세대원'])
house_policy = house_policy.T

#### gender

In [ ]:
u = len(user)
f = 2
p = len(policy)

In [ ]:
gender_policy = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
gender_policy.shape

(88486, 2)

In [ ]:
for idx, value in enumerate(policy.성별.to_numpy()):
  if '무관' in value:
    gender_policy[idx][0] = 1
    gender_policy[idx][1] = 1
    continue
  if '남성' in value:
    gender_policy[idx][0] = 1
  if '여성' in value:
    gender_policy[idx][1] = 1

In [ ]:
# gender_policy_df = pd.DataFrame(gender_policy, columns = ['남성', '여성'])
gender_policy = gender_policy.T

#### edu
고등학교졸업, 대학(원)재학, 대학(원)휴학, 대학(원)졸업 * policy

In [ ]:
u = len(user)
f = 4
p = len(policy)

In [ ]:
edu_policy = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
edu_policy.shape

(88486, 4)

In [ ]:
for idx, value in enumerate(policy.학력.to_numpy()):
  if '무관' in value:
    for i in range(f):
      edu_policy[idx][i] = 1
    continue
  if '고등학교졸업' in value:
    edu_policy[idx][0] = 1
  if '대학(원)재학' in value:
    edu_policy[idx][1] = 1
  if '대학(원)휴학' in value:
    edu_policy[idx][2] = 1
  if '대학(원)졸업' in value:
    edu_policy[idx][3] = 1

In [ ]:
# edu_policy_df = pd.DataFrame(edu_policy, columns = ['무관', '고등학교졸업', '대학(원)재학', '대학(원)휴학', '대학(원)졸업', '고등학교 졸업 미만'])
edu_policy = edu_policy.T

#### work
대학/대학원생, 구직자/실업자, 근로자/직장인, 예비창업자, 소상공인, 중소기업, 프리랜서 * policy

In [ ]:
u = len(user)
f = 7
p = len(policy)

In [ ]:
work_policy = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
work_policy.shape

(88486, 7)

In [ ]:
for idx, value in enumerate(policy.직장.to_numpy()):
  if '무관' in value:
    for i in range(f):
      work_policy[idx][i] = 1
    continue
  if '대학/대학원생' in value:
    work_policy[idx][0] = 1
  if '구직자/실업자' in value:
    work_policy[idx][1] = 1
  if '근로자/직장인' in value:
    work_policy[idx][2] = 1
  if '예비창업자' in value:
    work_policy[idx][3] = 1
  if '소상공인' in value:
    work_policy[idx][4] = 1
  if '중소기업' in value:
    work_policy[idx][5] = 1
  if '프리랜서' in value:
    work_policy[idx][6] = 1

In [ ]:
# work_policy_df = pd.DataFrame(work_policy, columns = ['무관', '대학/대학원생', '구직자/실업자', '근로자/직장인', '예비창업자', '소상공인', '중소기업', '프리랜서'])
work_policy = work_policy.T

#### 결혼
미혼, 기혼 * policy

In [ ]:
u = len(user)
f = 2
p = len(policy)

In [ ]:
marry_policy = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
marry_policy.shape

(88486, 2)

In [ ]:
for idx, value in enumerate(policy.결혼.to_numpy()):
  if ('무관' in value) or ('이혼' in value):
    marry_policy[idx][0] = 1
    marry_policy[idx][1] = 1
    continue
  if ('미혼' in value):
    marry_policy[idx][0] = 1
  if ('기혼' in value) or ('재혼' in value):
    marry_policy[idx][1] = 1

In [ ]:
# marry_policy_df = pd.DataFrame(marry_policy, columns = ['미혼', '기혼'])
marry_policy = marry_policy.T

#### city1
서울특별시, 부산광역시, 대구광역시, 인천광역시, 광주광역시, 대전광역시, 울산광역시, 세종특별자치시, 경기도, 강원도, 충청북도, 충청남도, 전라북도, 전라남도, 경상북도, 경상남도, 제주특별자치도 * policy

In [ ]:
u = len(user)
f = 17
p = len(policy)

In [ ]:
city_policy = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
city_policy.shape

(88486, 17)

In [ ]:
for idx, value in enumerate(policy['시도']):
  if '전국' in value: #전국
    for i in range(f):
      city_policy[idx][i] = 1
    continue
  
  city = ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도' ]
  for j, city_name in enumerate(city):
    if city_name in value:
      city_policy[idx][i] = 1

In [ ]:
# city_policy_df = pd.DataFrame(city_policy, columns = ['전국', '서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도'])
city_policy = city_policy.T

#### city2
종로구, 중구, 용산구, 성동구, 광진구, 동대문구, 중랑구, 성북구, 강북구, 도봉구, 노원구, 은평구, 서대문구, 마포구, 양천구, 강서구, 구로구, 금천구, 영등포구, 동작구, 관악구, 서초구, 강남구, 송파구, 강동구, 중구, 서구, 동구, 영도구, 부산진구, 동래구, 남구, 북구, 해운대구, 사하구, 금정구, 강서구, 연제구, 수영구, 사상구, 기장군, 중구, 동구, 서구, 남구, 북구, 수성구, 달서구, 달성군, 중구, 동구, 미추홀구, 연수구, 남동구, 부평구, 계양구, 서구, 강화군, 옹진군, 동구, 서구, 남구, 북구, 광산구, 동구, 중구, 서구, 유성구, 대덕구, 중구, 남구, 동구, 북구, 울주군, 세종시, 수원시, 성남시, 고양시, 용인시, 부천시, 안산시, 안양시, 남양주시, 화성시, 평택시, 의정부시, 시흥시, 파주시, 광명시, 김포시, 군포시, 광주시, 이천시, 양주시, 오산시, 구리시, 안성시, 포천시, 의왕시, 하남시, 여주시, 양평군, 동두천시, 과천시, 가평군, 연천군, 춘천시, 원주시, 강릉시, 동해시, 태백시, 속초시, 삼척시, 홍천군, 횡성군, 영월군, 평창군, 정선군, 철원군, 화천군, 양구군, 인제군, 고성군, 양양군, 청주시, 충주시, 제천시, 보은군, 옥천군, 영동군, 진천군, 괴산군, 음성군, 단양군, 증평군, 천안시, 공주시, 보령시, 아산시, 서산시, 논산시, 계룡시, 당진시, 금산군, 부여군, 서천군, 청양군, 홍성군, 예산군, 태안군, 전주시, 군산시, 익산시, 정읍시, 남원시, 김제시, 완주군, 진안군, 무주군, 장수군, 임실군, 순창군, 고창군, 부안군, 목포시, 여수시, 순천시, 나주시, 광양시, 담양군, 곡성군, 구례군, 고흥군, 보성군, 화순군, 장흥군, 강진군, 해남군, 영암군, 무안군, 함평군, 영광군, 장성군, 완도군, 진도군, 신안군, 포항시, 경주시, 김천시, 안동시, 구미시, 영주시, 영천시, 상주시, 문경시, 경산시, 군위군, 의성군, 청송군, 영양군, 영덕군, 청도군, 고령군, 성주군, 칠곡군, 예천군, 봉화군, 울진군, 울릉군, 창원시, 진주시, 통영시, 사천시, 김해시, 밀양시, 거제시, 양산시, 의령군, 함안군, 창녕군, 고성군, 남해군, 하동군, 산청군, 함양군, 거창군, 합천군, 제주시, 서귀포시 * policy

In [ ]:
city2 = ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', '중구', '서구', '동구', '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구', '금정구', '강서구', '연제구', '수영구', '사상구', '기장군', '중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군', '중구', '동구', '미추홀구', '연수구', '남동구', '부평구', '계양구', '서구', '강화군', '옹진군', '동구', '서구', '남구', '북구', '광산구', '동구', '중구', '서구', '유성구', '대덕구', '중구', '남구', '동구', '북구', '울주군', '세종시', '수원시', '성남시', '고양시', '용인시', '부천시', '안산시', '안양시', '남양주시', '화성시', '평택시', '의정부시', '시흥시', '파주시', '광명시', '김포시', '군포시', '광주시', '이천시', '양주시', '오산시', '구리시', '안성시', '포천시', '의왕시', '하남시', '여주시', '양평군', '동두천시', '과천시', '가평군', '연천군', '춘천시', '원주시', '강릉시', '동해시', '태백시', '속초시', '삼척시', '홍천군', '횡성군', '영월군', '평창군', '정선군', '철원군', '화천군', '양구군', '인제군', '고성군', '양양군', '청주시', '충주시', '제천시', '보은군', '옥천군', '영동군', '진천군', '괴산군', '음성군', '단양군', '증평군', '천안시', '공주시', '보령시', '아산시', '서산시', '논산시', '계룡시', '당진시', '금산군', '부여군', '서천군', '청양군', '홍성군', '예산군', '태안군', '전주시', '군산시', '익산시', '정읍시', '남원시', '김제시', '완주군', '진안군', '무주군', '장수군', '임실군', '순창군', '고창군', '부안군', '목포시', '여수시', '순천시', '나주시', '광양시', '담양군', '곡성군', '구례군', '고흥군', '보성군', '화순군', '장흥군', '강진군', '해남군', '영암군', '무안군', '함평군', '영광군', '장성군', '완도군', '진도군', '신안군', '포항시', '경주시', '김천시', '안동시', '구미시', '영주시', '영천시', '상주시', '문경시', '경산시', '군위군', '의성군', '청송군', '영양군', '영덕군', '청도군', '고령군', '성주군', '칠곡군', '예천군', '봉화군', '울진군', '울릉군', '창원시', '진주시', '통영시', '사천시', '김해시', '밀양시', '거제시', '양산시', '의령군', '함안군', '창녕군', '고성군', '남해군', '하동군', '산청군', '함양군', '거창군', '합천군', '제주시', '서귀포시']
u = len(user)
f = len(city2)
p = len(policy)

In [ ]:
city2_policy = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
city2_policy.shape

(88486, 229)

In [ ]:
for idx, value in enumerate(policy['시군구']):
  if ('' == value) or (value == None):
    # for i in range(f):
    #   city2_policy[idx][i] = 1
    continue
  
  for j, city_name in enumerate(city2):
    if city_name in value:
      city2_policy[idx][i] = 1

In [ ]:
# city2_policy_df = pd.DataFrame(city2_policy)
city2_policy = city2_policy.T

#### money
기초생활수급자, 중위소득 40% 이하, 중위소득 40 ~ 60% 사이, 중위소득 60 ~ 80% 사이, 중위소득 80 ~ 100% 사이, 중위소득 100% 이상 * policy

In [ ]:
mb_12 = ['기초생활수급자', '중위소득 40% 이하', '중위소득 40~60% 사이', '중위소득 60~80% 사이', '중위소득 80~100% 사이',  '중위소득 100% 이상']
u = len(user)
f = len(mb_12)
p = len(policy)

In [ ]:
money_policy = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
money_policy.shape

(88486, 6)

In [ ]:
for idx, value in enumerate(policy.중위소득.to_numpy()):
  if ('' == value) or (value == None):
    for i in range(f):
      money_policy[idx][i] = 1
    continue
  if '기초생활수급자' in value:
    money_policy[idx][0] = 1
  if '30' in value:
    money_policy[idx][1] = 1
  if '50' in value:
    money_policy[idx][2] = 1
  if '70' in value:
    money_policy[idx][3] = 1
  if '90' in value:
    money_policy[idx][4] = 1
  if '110' in value:
    money_policy[idx][5] = 1

In [ ]:
# money_policy_df = pd.DataFrame(money_policy, columns = ['무관', '대학/대학원생', '구직자/실업자', '근로자/직장인', '예비창업자', '소상공인', '중소기업', '프리랜서'])
money_policy = money_policy.T

#### mb_15
근로자 지원, 취업 지원, 개인금융지원, 문화생활 지원, 주택-부동산 지원, 교육지원(만8 ~ 19세), 성인교육지원, 의료 지원, 창업 지원, 기업금융지원, 보육지원(만0~7세) * policy

In [ ]:
mb_15 = ['근로자 지원', '취업 지원', '개인금융지원', '문화생활 지원', '주택-부동산 지원', '교육지원(만8~19세)', '성인교육지원',
         '의료 지원', '창업 지원', '기업금융지원', '보육지원(만0~7세)']

u = len(user)
f = len(mb_15)
p = len(policy)

In [ ]:
def mb_15_edit(text):
  if '취업지원' in text:
    text = text.replace('취업지원', '취업 지원')
  return text

policy.MB_15 = policy.MB_15.apply(mb_15_edit)


In [ ]:
print(len(get_category(policy,'MB_15')))
get_category(policy,'MB_15')

13


[('개인금융지원', 57540),
 ('기업금융지원', 30910),
 ('의료 지원', 21132),
 ('창업 지원', 21094),
 ('근로자 지원', 12114),
 ('문화생활 지원', 9410),
 ('', 8959),
 ('취업 지원', 6571),
 ('보육지원(0~7세)', 4674),
 ('교육지원(8~19세)', 3298),
 ('성인교육지원', 3288),
 ('주택-부동산 지원', 3027),
 '중복 있음']

In [ ]:
mb_15_policy = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
mb_15_policy.shape

(88486, 11)

In [ ]:
for idx, value in enumerate(policy.MB_15.to_numpy()):
  if '' == value:
    for i in range(f):
      mb_15_policy[idx][i] = 1
    continue
  for i, text in enumerate(mb_15):
    if text in value:
      word_idx = mb_15.index(text)
      mb_15_policy[idx][word_idx] = 1

In [ ]:
# mb_15_policy_df = pd.DataFrame(mb_15_policy, columns = ['무관', '대학/대학원생', '구직자/실업자', '근로자/직장인', '예비창업자', '소상공인', '중소기업', '프리랜서'])
mb_15_policy = mb_15_policy.T

### feature * policy matrix 11개 for 특수

#### 카테고리별 갯수 뽑아오는 함수

In [ ]:
def get_category(df, columns):
  data = df[columns]
  category = {}
  multi = False
  for word in data:
    lst = []
    if word == None:
      lst.append(None)
    elif ',' in word:
      lst = [x.strip() for x in word.split(',')]
    else:
      lst.append(word)
    for i in lst:
      if i in category:
        category[i] += 1
      else:
        category[i] = 1
    if ',' in word:
      multi = True

  category = list(sorted(category.items(), key=lambda item: item[1], reverse=True))
  if multi == True:
    category.append('중복 있음')
  else:
    category.append('중복 없음')
  return category

#### child

In [ ]:
u = len(user)
f = 2
p = len(policy)

In [ ]:
child_policy2 = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
child_policy2.shape

(88486, 2)

In [ ]:
for idx, value in enumerate(policy.자녀.to_numpy()):
  if value== '없음': # 없음
    child_policy2[idx][0] = 1
  if value== '있음': # 있음
    child_policy2[idx][1] = 1

In [ ]:
# child_policy2_df = pd.DataFrame(child_policy2, columns=['없음', '있음'])
child_policy2 = child_policy2.T

#### child_detail

In [ ]:
u = len(user)
f = 5
p = len(policy)

In [ ]:
child_detail_policy2 = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
child_detail_policy2.shape

(88486, 5)

In [ ]:
get_category(policy, '자녀상세')

[('없음', 78623),
 ('영유아', 8189),
 ('초중고 학생', 6306),
 ('어린이집', 2718),
 ('유치원', 2659),
 ('성인', 2518),
 '중복 있음']

In [ ]:
for idx, value in enumerate(policy.자녀상세.to_numpy()):
  if (value== '해당사항없음'):
    # for i in range(5):
    #   child_detail_policy2[idx][i] = 0
    continue
  if '영유아' in value:
    child_detail_policy2[idx][0] = 1
  if '유치원' in value:
    child_detail_policy2[idx][1] = 1
  if '어린이집' in value:
    child_detail_policy2[idx][2] = 1
  if '초중고학생' in value:
    child_detail_policy2[idx][3] = 1
  if '성인' in value:
    child_detail_policy2[idx][4] = 1

In [ ]:
# child_detail_policy2_df = pd.DataFrame(child_detail_policy2, columns = ['영유아', '유치원', '어린이집', '초중고학생', '성인'])
child_detail_policy2 = child_detail_policy2.T

#### house

In [ ]:
u = len(user)
f = 4
p = len(policy)

In [ ]:
house_policy2 = np.array([ [ 0 for i in range(f) ] for j in range(p)])
house_policy2.shape

(88486, 4)

In [ ]:
for idx, value in enumerate(policy.가구원.to_numpy()):
  if '무관' in value:
    for i in range(4):
      house_policy2[idx][i] = 1
    continue
  if '무주택 세대주' in value:
    house_policy2[idx][0] = 1
  if '무주택 세대원' in value:
    house_policy2[idx][1] = 1
  if '주택 세대주' in value:
    house_policy2[idx][2] = 1
  if '주택 세대원' in value:
    house_policy2[idx][3] = 1

In [ ]:
# house_policy2_df = pd.DataFrame(house_policy2, columns = ['무관', '무주택 세대주', '무주택 세대원', '주택 세대주', '주택 세대원'])
house_policy2 = house_policy2.T

#### gender

In [ ]:
u = len(user)
f = 2
p = len(policy)

In [ ]:
gender_policy2 = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
gender_policy2.shape

(88486, 2)

In [ ]:
for idx, value in enumerate(policy.성별.to_numpy()):
  if '무관' in value:
    gender_policy2[idx][0] = 1
    gender_policy2[idx][1] = 1
    continue
  if '남성' in value:
    gender_policy2[idx][0] = 1
  if '여성' in value:
    gender_policy2[idx][1] = 1

In [ ]:
# gender_policy2_df = pd.DataFrame(gender_policy2, columns = ['남성', '여성'])
gender_policy2 = gender_policy2.T

#### edu
고등학교졸업, 대학(원)재학, 대학(원)휴학, 대학(원)졸업 * policy

In [ ]:
u = len(user)
f = 4
p = len(policy)

In [ ]:
edu_policy2 = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
edu_policy2.shape

(88486, 4)

In [ ]:
for idx, value in enumerate(policy.학력.to_numpy()):
  if '무관' in value:
    # for i in range(f):
    #   edu_policy2[idx][i] = 1
    continue
  if '고등학교졸업' in value:
    edu_policy2[idx][0] = 1
  if '대학(원)재학' in value:
    edu_policy2[idx][1] = 1
  if '대학(원)휴학' in value:
    edu_policy2[idx][2] = 1
  if '대학(원)졸업' in value:
    edu_policy2[idx][3] = 1

In [ ]:
# edu_policy2_df = pd.DataFrame(edu_policy2, columns = ['무관', '고등학교졸업', '대학(원)재학', '대학(원)휴학', '대학(원)졸업', '고등학교 졸업 미만'])
edu_policy2 = edu_policy2.T

#### work
대학/대학원생, 구직자/실업자, 근로자/직장인, 예비창업자, 소상공인, 중소기업, 프리랜서 * policy

In [ ]:
u = len(user)
f = 7
p = len(policy)

In [ ]:
work_policy2 = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
work_policy2.shape

(88486, 7)

In [ ]:
for idx, value in enumerate(policy.직장.to_numpy()):
  if '무관' in value:
    # for i in range(f):
    #   work_policy2[idx][i] = 1
    continue
  if '대학/대학원생' in value:
    work_policy2[idx][0] = 1
  if '구직자/실업자' in value:
    work_policy2[idx][1] = 1
  if '근로자/직장인' in value:
    work_policy2[idx][2] = 1
  if '예비창업자' in value:
    work_policy2[idx][3] = 1
  if '소상공인' in value:
    work_policy2[idx][4] = 1
  if '중소기업' in value:
    work_policy2[idx][5] = 1
  if '프리랜서' in value:
    work_policy2[idx][6] = 1

In [ ]:
# work_policy2_df = pd.DataFrame(work_policy2, columns = ['무관', '대학/대학원생', '구직자/실업자', '근로자/직장인', '예비창업자', '소상공인', '중소기업', '프리랜서'])
work_policy2 = work_policy2.T

#### 결혼
미혼, 기혼 * policy

In [ ]:
u = len(user)
f = 2
p = len(policy)

In [ ]:
marry_policy2 = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
marry_policy2.shape

(88486, 2)

In [ ]:
for idx, value in enumerate(policy.결혼.to_numpy()):
  if ('무관' in value) or ('이혼' in value):
    marry_policy2[idx][0] = 0
    marry_policy2[idx][1] = 0
    continue
  if ('미혼' in value):
    marry_policy2[idx][0] = 1
  if ('기혼' in value) or ('재혼' in value):
    marry_policy2[idx][1] = 1

In [ ]:
# marry_policy2_df = pd.DataFrame(marry_policy2, columns = ['미혼', '기혼'])
marry_policy2 = marry_policy2.T

#### city1
서울특별시, 부산광역시, 대구광역시, 인천광역시, 광주광역시, 대전광역시, 울산광역시, 세종특별자치시, 경기도, 강원도, 충청북도, 충청남도, 전라북도, 전라남도, 경상북도, 경상남도, 제주특별자치도 * policy

In [ ]:
u = len(user)
f = 17
p = len(policy)

In [ ]:
city_policy2 = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
city_policy2.shape

(88486, 17)

In [ ]:
for idx, value in enumerate(policy['시도']):
  if '전국' in value: #전국
    for i in range(f):
      city_policy2[idx][i] = 1
    continue
  
  city = ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도' ]
  for j, city_name in enumerate(city):
    if city_name in value:
      city_policy2[idx][i] = 1

In [ ]:
# city_policy2_df = pd.DataFrame(city_policy2, columns = ['전국', '서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도'])
city_policy2 = city_policy2.T

#### city2
종로구, 중구, 용산구, 성동구, 광진구, 동대문구, 중랑구, 성북구, 강북구, 도봉구, 노원구, 은평구, 서대문구, 마포구, 양천구, 강서구, 구로구, 금천구, 영등포구, 동작구, 관악구, 서초구, 강남구, 송파구, 강동구, 중구, 서구, 동구, 영도구, 부산진구, 동래구, 남구, 북구, 해운대구, 사하구, 금정구, 강서구, 연제구, 수영구, 사상구, 기장군, 중구, 동구, 서구, 남구, 북구, 수성구, 달서구, 달성군, 중구, 동구, 미추홀구, 연수구, 남동구, 부평구, 계양구, 서구, 강화군, 옹진군, 동구, 서구, 남구, 북구, 광산구, 동구, 중구, 서구, 유성구, 대덕구, 중구, 남구, 동구, 북구, 울주군, 세종시, 수원시, 성남시, 고양시, 용인시, 부천시, 안산시, 안양시, 남양주시, 화성시, 평택시, 의정부시, 시흥시, 파주시, 광명시, 김포시, 군포시, 광주시, 이천시, 양주시, 오산시, 구리시, 안성시, 포천시, 의왕시, 하남시, 여주시, 양평군, 동두천시, 과천시, 가평군, 연천군, 춘천시, 원주시, 강릉시, 동해시, 태백시, 속초시, 삼척시, 홍천군, 횡성군, 영월군, 평창군, 정선군, 철원군, 화천군, 양구군, 인제군, 고성군, 양양군, 청주시, 충주시, 제천시, 보은군, 옥천군, 영동군, 진천군, 괴산군, 음성군, 단양군, 증평군, 천안시, 공주시, 보령시, 아산시, 서산시, 논산시, 계룡시, 당진시, 금산군, 부여군, 서천군, 청양군, 홍성군, 예산군, 태안군, 전주시, 군산시, 익산시, 정읍시, 남원시, 김제시, 완주군, 진안군, 무주군, 장수군, 임실군, 순창군, 고창군, 부안군, 목포시, 여수시, 순천시, 나주시, 광양시, 담양군, 곡성군, 구례군, 고흥군, 보성군, 화순군, 장흥군, 강진군, 해남군, 영암군, 무안군, 함평군, 영광군, 장성군, 완도군, 진도군, 신안군, 포항시, 경주시, 김천시, 안동시, 구미시, 영주시, 영천시, 상주시, 문경시, 경산시, 군위군, 의성군, 청송군, 영양군, 영덕군, 청도군, 고령군, 성주군, 칠곡군, 예천군, 봉화군, 울진군, 울릉군, 창원시, 진주시, 통영시, 사천시, 김해시, 밀양시, 거제시, 양산시, 의령군, 함안군, 창녕군, 고성군, 남해군, 하동군, 산청군, 함양군, 거창군, 합천군, 제주시, 서귀포시 * policy

In [ ]:
city2 = ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', '중구', '서구', '동구', '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구', '금정구', '강서구', '연제구', '수영구', '사상구', '기장군', '중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군', '중구', '동구', '미추홀구', '연수구', '남동구', '부평구', '계양구', '서구', '강화군', '옹진군', '동구', '서구', '남구', '북구', '광산구', '동구', '중구', '서구', '유성구', '대덕구', '중구', '남구', '동구', '북구', '울주군', '세종시', '수원시', '성남시', '고양시', '용인시', '부천시', '안산시', '안양시', '남양주시', '화성시', '평택시', '의정부시', '시흥시', '파주시', '광명시', '김포시', '군포시', '광주시', '이천시', '양주시', '오산시', '구리시', '안성시', '포천시', '의왕시', '하남시', '여주시', '양평군', '동두천시', '과천시', '가평군', '연천군', '춘천시', '원주시', '강릉시', '동해시', '태백시', '속초시', '삼척시', '홍천군', '횡성군', '영월군', '평창군', '정선군', '철원군', '화천군', '양구군', '인제군', '고성군', '양양군', '청주시', '충주시', '제천시', '보은군', '옥천군', '영동군', '진천군', '괴산군', '음성군', '단양군', '증평군', '천안시', '공주시', '보령시', '아산시', '서산시', '논산시', '계룡시', '당진시', '금산군', '부여군', '서천군', '청양군', '홍성군', '예산군', '태안군', '전주시', '군산시', '익산시', '정읍시', '남원시', '김제시', '완주군', '진안군', '무주군', '장수군', '임실군', '순창군', '고창군', '부안군', '목포시', '여수시', '순천시', '나주시', '광양시', '담양군', '곡성군', '구례군', '고흥군', '보성군', '화순군', '장흥군', '강진군', '해남군', '영암군', '무안군', '함평군', '영광군', '장성군', '완도군', '진도군', '신안군', '포항시', '경주시', '김천시', '안동시', '구미시', '영주시', '영천시', '상주시', '문경시', '경산시', '군위군', '의성군', '청송군', '영양군', '영덕군', '청도군', '고령군', '성주군', '칠곡군', '예천군', '봉화군', '울진군', '울릉군', '창원시', '진주시', '통영시', '사천시', '김해시', '밀양시', '거제시', '양산시', '의령군', '함안군', '창녕군', '고성군', '남해군', '하동군', '산청군', '함양군', '거창군', '합천군', '제주시', '서귀포시']
u = len(user)
f = len(city2)
p = len(policy)

In [ ]:
city2_policy2 = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
city2_policy2.shape

(88486, 229)

In [ ]:
for idx, value in enumerate(policy['시군구']):
  if ('' == value) or  (value == None):
    # for i in range(f):
    #   city2_policy2[idx][i] = 1
    continue
  
  for j, city_name in enumerate(city2):
    if city_name in value:
      city2_policy2[idx][i] = 1

In [ ]:
# city2_policy2_df = pd.DataFrame(city2_policy2)
city2_policy2 = city2_policy2.T

#### money
기초생활수급자, 중위소득 40% 이하, 중위소득 40 ~ 60% 사이, 중위소득 60 ~ 80% 사이, 중위소득 80 ~ 100% 사이, 중위소득 100% 이상 * policy

In [ ]:
mb_12 = ['기초생활수급자', '중위소득 40% 이하', '중위소득 40~60% 사이', '중위소득 60~80% 사이', '중위소득 80~100% 사이',  '중위소득 100% 이상']
u = len(user)
f = len(mb_12)
p = len(policy)

In [ ]:
money_policy2 = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
money_policy2.shape

(88486, 6)

In [ ]:
for idx, value in enumerate(policy.중위소득.to_numpy()):
  if ('' == value) or (value == None):
    # for i in range(f):
    #   money_policy2[idx][i] = 1
    continue
  if '기초생활수급자' in value:
    money_policy2[idx][0] = 1
  if '30' in value:
    money_policy2[idx][1] = 1
  if '50' in value:
    money_policy2[idx][2] = 1
  if '70' in value:
    money_policy2[idx][3] = 1
  if '90' in value:
    money_policy2[idx][4] = 1
  if '110' in value:
    money_policy2[idx][5] = 1

In [ ]:
# money_policy2_df = pd.DataFrame(money_policy2, columns = ['무관', '대학/대학원생', '구직자/실업자', '근로자/직장인', '예비창업자', '소상공인', '중소기업', '프리랜서'])
money_policy2 = money_policy2.T

#### mb_15
근로자 지원, 취업 지원, 개인금융지원, 문화생활 지원, 주택-부동산 지원, 교육지원(만8 ~ 19세), 성인교육지원, 의료 지원, 창업 지원, 기업금융지원, 보육지원(만0~7세) * policy

In [ ]:
mb_15 = ['근로자 지원', '취업 지원', '개인금융지원', '문화생활 지원', '주택-부동산 지원', '교육지원(만8~19세)', '성인교육지원',
         '의료 지원', '창업 지원', '기업금융지원', '보육지원(만0~7세)']

u = len(user)
f = len(mb_15)
p = len(policy)

In [ ]:
def mb_15_edit(text):
  if '취업지원' in text:
    text = text.replace('취업지원', '취업 지원')
  return text

policy.MB_15 = policy.MB_15.apply(mb_15_edit)


In [ ]:
print(len(get_category(policy,'MB_15')))
get_category(policy,'MB_15')

13


[('개인금융지원', 57540),
 ('기업금융지원', 30910),
 ('의료 지원', 21132),
 ('창업 지원', 21094),
 ('근로자 지원', 12114),
 ('문화생활 지원', 9410),
 ('', 8959),
 ('취업 지원', 6571),
 ('보육지원(0~7세)', 4674),
 ('교육지원(8~19세)', 3298),
 ('성인교육지원', 3288),
 ('주택-부동산 지원', 3027),
 '중복 있음']

In [ ]:
mb_15_policy2 = np.array([ [ 0 for i in range(f) ] for j in range(p) ])
mb_15_policy2.shape

(88486, 11)

In [ ]:
for idx, value in enumerate(policy.MB_15.to_numpy()):
  if '' == value:
    # for i in range(f):
    #   mb_15_policy2[idx][i] = 1
    continue
  for i, text in enumerate(mb_15):
    if text in value:
      word_idx = mb_15.index(text)
      mb_15_policy2[idx][word_idx] = 1

In [ ]:
# mb_15_policy2_df = pd.DataFrame(mb_15_policy2, columns = ['무관', '대학/대학원생', '구직자/실업자', '근로자/직장인', '예비창업자', '소상공인', '중소기업', '프리랜서'])
mb_15_policy2 = mb_15_policy2.T

## 유사도

#### 보편 모델

In [ ]:
user_feature_matrix = [gender_matrix, 'age', city_matrix, city2_matrix, edu_matrix, work_matrix, house_matrix, marry_matrix, child_matrix, child_detail_matrix, money_matrix, mb_15_matrix]

In [ ]:
feature_policy_matrix = [gender_policy, 'age',  city_policy, city2_policy, edu_policy, work_policy, house_policy, marry_policy, child_policy, child_detail_policy, money_policy, mb_15_policy]

In [ ]:
# array를 minmax scaler로 조정
def minmax(array):
  a, b = array.shape
  change = array.reshape(-1,1)
  minmax = MinMaxScaler()
  change = minmax.fit_transform(change)
  change = change.reshape(a,b)
  return change

In [ ]:
# user_info에 직접 데이터를 넣으려면 성별, 나이, 시도, 시 군구, 학력, 직장, 가구원, 결혼, 자녀, 자녀상세, mb_15순서이 list 형태로 넣어야 함
# None 대신 '' 넣기

def recommendation(user_info):
  # user 정보 보여주기
  if type(user_info) == int:
    print(f'===================={user_info}번째 user 정보=================')
    for i in user.columns:
      print(f'{i}: ', end='')
      print(user[i][user_info])
    print()
    user_info = user[['성별', '나이', '시도', '시군구', '학력', '직장', '가구원', '결혼', '자녀', '자녀상세', 'mb_12', 'mb_15']].iloc[user_info]
  elif (type(user_info) == list) or (type(user_info) == numpy.ndarray):
    print(f'===================user 정보=================')
    for i, column in enumerate(['성별', '나이', '시도', '시군구', '학력', '직장', '가구원', '결혼', '자녀', '자녀상세', '중위소득', '관심정책']):
      print(f'{column}: ', end='')
      print(user_info[i])
    print()
  else:
    print('user_info에 직접 데이터를 넣으려면 [성별, 나이, 시도, 시군구, 학력, 직장, 가구원, 결혼, 자녀, 자녀상세, 중위소득, 관심정책] 순서의 리스트 형태로 넣어야 함')
    return False
  
  # 유사도 매트릭스 구하기
  ans = np.array([[0 for i in range(len(policy))] for j in range(1)])
  for i, value in enumerate(user_info):
    if i == 1:
      ans += age_matrix(value)
    elif (i == 6) or (i == 9):
      ans = ans + minmax(np.dot(user_feature_matrix[i](value), feature_policy_matrix[i])) * 0.5
    elif (i == 2) or (i==8) or (i==5):
      ans = ans + minmax(np.dot(user_feature_matrix[i](value), feature_policy_matrix[i])) * 2
    elif (i == 10) or (i == 11):
      ans = ans + minmax(np.dot(user_feature_matrix[i](value), feature_policy_matrix[i])) * 3
    else:
      ans += np.dot(user_feature_matrix[i](value), feature_policy_matrix[i])
    
  # 추천하는 policy 정보 보여주기
  best = max(ans[0])
  best_idxs = [i for i, v in enumerate(ans[0]) if v == best]
  print(f'{len(best_idxs)}개가 {best}값으로 최적의 정책임')

  for best_idx in best_idxs:
    print('=====================policy 정보=================')
    for column in policy.columns:
      print(f'◎{column}: ', end='')
      print(policy[column].iloc[best_idx])
    print()

  print(f'{len(best_idxs)}개가 {best}값으로 최적의 정책임')

In [ ]:
recommendation(['여성', 27, '경기도, 서울', '파주시', '대학(원)졸업', '', '', '미혼', '없음', '', '', ''])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

○ 특허분석 가이드
 - 특허검색 및 분석 방법, 특허 DB, 특허분석 Tool 가이드 제공
 - 주요 국가별 특허제도 및 관련 홈페이지 소개
 - 특허검색 실습 서비스 제공



금융·세금·법률>법률/특허>특허||결혼·육아·교육>기술개발/학술연구>기술개발
◎지원대상: ○ 대국민

일반
◎대상연령시작: 6
◎대상연령끝: 99
◎시도: 전국
◎시군구: None
◎중위소득: None
◎관심상황특성: 
◎대상특성: 해당사항없음
◎대상특성상세: 
◎MB_15: 
◎소관기관유형: 공공기관 등
◎지원유형: 기타
◎지원유형상세: 기타(상담)
◎신청절차: 무관
◎성별: 무관
◎결혼: 무관
◎학력: 무관
◎가구원: 무관
◎자녀: 없음
◎자녀상세: 없음
◎직장: 무관
◎직장상세: None

=====================policy 정보=================
◎정책ID: 86848
◎정책서비스ID: B55190100038
◎서비스명: 나눔 문화
◎소관기관: 한국보건복지인력개발원
◎소관기관유형.1: 공공기관 등
◎생애주기: 영·유아||아동·청소년||청년||중장년||노년기
◎정책신청절차: 온라인신청('수강신청'클릭)
◎선정기준: None
◎정책지원유형: 봉사/기부하기

교육/용역
◎서비스목적: ∙ 나눔의 의미와 필요성에 대해 알고 나눔 확산 정책을 이해할 수 있다.
∙ 나눔 확산을 위한 민간자원을 이해하고 나눔 확산 활동에 참여할 수 있다.
◎지원내용: 나눔 정책의 이해 및 방향, 나눔의 실천, 공정한 분배와 나눔

20180102~20181205 (기타:상황에 따라 조기 종료 가능)

공익·봉사>사회봉사>봉사활동
◎지원대상: ∙ 보건복지 관련 공무원 및 일반인

일반
◎대상연령시작: 0
◎대상연령끝: 99
◎시도: 전국
◎시군구: None
◎중위소득: None
◎관심상황특성: 
◎대상특성: 해당사항없음
◎대상특성상세: 
◎MB_15: 
◎소관기관유형: 공공기관 등
◎지원유형: 봉사/기부
◎지원유형상

In [ ]:
recommendation(['여성', 27, '경기도', '파주시', '대학(원)졸업', '구직자/실업자', '', '미혼', '없음', None, '중위소득 40% 이하, 중위소득 40 ~ 60% 사이','취업 지원'])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 

○ 제출 기한 : ‘18년 8월 9일(목) 18:00까지(제출기한 엄수)

 

○ 접수방법 : 우편(마감일까지 도착분에 한함) 또는 방문접수 가능
◎선정기준: None
◎정책지원유형: 현금/현물
◎서비스목적: ○ 고졸자의 진학 욕구 및 조기 취업과 중소·중견기업에 맞춤형 인재양성·공급과 기술경쟁력 강화를 동시에 달성하는 청년일자리 교육모델 마련
◎지원내용: ○ 사업 기간 : ‘18년～’23년 (준비기간 1년 포함 4년+2년)

※ 대학의 책무성 제고를 위해 4년 지원 후 단계평가를 통해 계속지원 여부 결정

 

○ 사업 예산 : ‘18년 80억원

- 총 80억원 / 대학당 평균 16억원 내외/ 5개교 내외 선정

※ 사업내용, 선정평가 결과 등을 고려하여 대학별 지원금 차등 지원 가능

 

○ 지원 대상 : 총 5개교 내외

- 1단계 서면평가(정량 및 정성)를 통해 2배수(권역별 2개교 내외, 총 10개교 내외)를 선정하고 2단계 심층 대면평가를 통해 5개교 내외 선정

20180619~20180809 (기타:※ 제출된 서류는 변경할 수 없으며, 반환하지 않음)

창업·경영>업종별민원>전문업/과학업/기술업
◎지원대상: ○ (신청대상) 고등교육법 제2조 제1호, 2호에 해당하는 대학, 산업대학

※ 2018년 재정지원제한대학은 참여는 허용되나, 사업 선정 시 원칙적으로 재정지원 제한기간 동안 재정지원은 불가하며, 대학 자체 재원으로 사업 수행

 

○ (신청단위) 사업단

- 분교의 경우, 산학협력단 법인을 본교와 분리하여 별도로 운영하는 대학에 한해 지원가능하며 분교 소재지가 위치한 권역에 신청*

* 분교 설립인가를 받지 않은 캠퍼스에 속하는 사업단은 본교와 통합하여 본교 소재지가 위치한 권역에 신청

기업인||중소기업/소상공인
◎대상연령시작: 19
◎대상연령끝: 99
◎시도: 전국
◎시군구: None
◎중위소득: None
◎관심상황특성: 
◎대상특성: 해당사항없음
◎대상특성상세

#### 특수모델

In [ ]:
user_feature_matrix2 = [gender_matrix2, 'age', city_matrix2, city2_matrix2, edu_matrix2, work_matrix2, house_matrix2, marry_matrix2, child_matrix2, child_detail_matrix2, money_matrix2, mb_15_matrix2]

In [ ]:
feature_policy_matrix2 = [gender_policy2, 'age',  city_policy2, city2_policy2, edu_policy2, work_policy2, house_policy2, marry_policy2, child_policy2, child_detail_policy2, money_policy2, mb_15_policy]

In [ ]:
# user_info에 직접 데이터를 넣으려면 성별, 나이, 시도, 시 군구, 학력, 직장, 가구원, 결혼, 자녀, 자녀상세, mb_15순서이 list 형태로 넣어야 함

def recommendation2(user_info):
  # user 정보 보여주기
  if type(user_info) == int:
    print(f'===================={user_info}번째 user 정보=================')
    for i in user.columns:
      print(f'{i}: ', end='')
      print(user[i][user_info])
    print()
    user_info = user[['성별', '나이', '시도', '시군구', '학력', '직장', '가구원', '결혼', '자녀', '자녀상세', 'mb_12', 'mb_15']].iloc[user_info]
  elif (type(user_info) == list) or (type(user_info) == numpy.ndarray):
    print(f'===================user 정보=================')
    for i, column in enumerate(['성별', '나이', '시도', '시군구', '학력', '직장', '가구원', '결혼', '자녀', '자녀상세', '중위소득', '관심정책']):
      print(f'{column}: ', end='')
      print(user_info[i])
    print()
  else:
    print('user_info에 직접 데이터를 넣으려면 [성별, 나이, 시도, 시군구, 학력, 직장, 가구원, 결혼, 자녀, 자녀상세, 중위소득, 관심정책] 순서의 리스트 형태로 넣어야 함')
    return False
  
  # 유사도 매트릭스 구하기
  ans = np.array([[0 for i in range(len(policy))] for j in range(1)])
  for i, value in enumerate(user_info):
    if i == 1:
      ans += age_matrix(value)
    elif (i == 6) or (i == 9):
      ans = ans + minmax(np.dot(user_feature_matrix2[i](value), feature_policy_matrix2[i])) * 0.5
    elif (i == 2) or (i==8) or (i==5):
      ans = ans + minmax(np.dot(user_feature_matrix2[i](value), feature_policy_matrix2[i])) * 2
    elif (i == 10) or (i == 11):
      ans = ans + minmax(np.dot(user_feature_matrix2[i](value), feature_policy_matrix2[i])) * 3
    else:
      ans += np.dot(user_feature_matrix2[i](value), feature_policy_matrix2[i])
    
  # 추천하는 policy 정보 보여주기
  best = max(ans[0])
  best_idxs = [i for i, v in enumerate(ans[0]) if v == best]
  print(f'{len(best_idxs)}개가 {best}값으로 최적의 정책임')

  for best_idx in best_idxs:
    print('=====================policy 정보=================')
    print(best_idx)
    for column in policy.columns:
      print(f'◎{column}: ', end='')
      print(policy[column].iloc[best_idx])
    print()

  print(f'{len(best_idxs)}개가 {best}값으로 최적의 정책임')

In [ ]:
recommendation2(['여성', 27, '경기도', '파주시', '대학(원)졸업', '구직자/실업자', '', '미혼', '없음', None, '중위소득 40% 이하, 중위소득 40 ~ 60% 사이','취업 지원'])

===================user 정보=================
성별: 여성
나이: 27
시도: 경기도
시군구: 파주시
학력: 대학(원)졸업
직장: 구직자/실업자
가구원: 
결혼: 미혼
자녀: 없음
자녀상세: None
중위소득: 중위소득 40% 이하, 중위소득 40 ~ 60% 사이
관심정책: 취업 지원

8개가 14.0값으로 최적의 정책임
=====================policy 정보=================
389
◎정책ID: 390
◎정책서비스ID: 149200000037
◎서비스명: 직업훈련생계비 대부
◎소관기관: 고용노동부
◎소관기관유형.1: 중앙정부
◎생애주기: None
◎정책신청절차: 근로복지공단에 방문하거나 온라인으로 신청
◎선정기준: ○ 고용노동부가 지원하는 훈련 중 3주 이상 훈련에 참여하고 있는 비정규직 근로자 및 전직실업자, 무급휴직자, 특수형태근로종사자, 휴폐업한 자영업자 중 가구소득이 기준중위소득의 100% 이하인 자
◎정책지원유형: 현금(융자)
◎서비스목적: 취약계층(비정규직근로자, 전직실업자, 무급휴직자, 특수형태근로종사자, 휴폐업한 자영업자)이 생계비에 대한 부담 없이 장기간 체계적인 훈련을 받고 더 나은 일자리로의 취업을 할 수 있도록 생계비를 융자 신청할 수 있도록 함
◎지원내용: ○ 1인당 대부 한도액 : 2,000만 원 (특별고용지원업종 및 고용위기지역은 3,000만원)

○ 월별 대부 한도액 : 200만 원(최소 50만 원) 

○ 상환 방법 : 1년 거치 3년, 2년 거치 4년, 3년 거치 5년 균등분할 상환 

○ 융자금리 : 연 1% 

○ 최대 3년 거치 5년 균등 상환
◎지원대상: ○ 3주 이상인 직업훈련*에 참여하는 비정규직 근로자, 전직실업자, 무급휴직자, 특수형태근로종사자(산재보험 가입자에 한함), 휴폐업한 자영업자 중 가구원 합산 소득이 기준 중위소득의 100% 이하인 자
 - 전직실업자인 경우 실업급여 수급중인 자는 제외
  · 직업훈련: 고용 노동부에서 인정하는 훈련과정(실업

In [ ]:
recommendation2(['여성', 27, '경기도, 서울', '파주시', '대학(원)졸업', '', '', '미혼', '없음', '', '', ''])

===================user 정보=================
성별: 여성
나이: 27
시도: 경기도, 서울
시군구: 파주시
학력: 대학(원)졸업
직장: 
가구원: 
결혼: 미혼
자녀: 없음
자녀상세: 
중위소득: 
관심정책: 

22개가 7.0값으로 최적의 정책임
=====================policy 정보=================
126
◎정책ID: 127
◎정책서비스ID: 134200000008
◎서비스명: 글로벌 현장학습 프로그램 운영
◎소관기관: 교육부
◎소관기관유형.1: 중앙정부
◎생애주기: None
◎정책신청절차: ○ 대학/전문대학 글로벌현장학습, WEST, 파란사다리 학생 모집 공고를 확인하여 사업별 신청절차 확인 필요
  - 세부사항은 사업시행기관에 문의 가능 :(2020 시행기관) 대학 글로벌 - 한국장학재단(053-238-2972), 파란사다리 - 한국장학재단(053-238-2974), 전문대학글로벌 - 한국전문대학교육협의회(02-3145-1252), WEST- 국립국제교육원(02-3668-1307)
 *관련 사이트 : 월드 잡플러스(해외통합정보망) - 해외인턴 Q&A
◎선정기준: ○ 대학 글로벌 현장학습(대학 단위 모집)
 - 4학기 이상 수료자로 누계 평점이 3.0/4.5 이상인 재·휴학생으로 일정 기준 이상의 어학 능력을 갖추어 소속 대학을 통해 선정

○ 전문대학 글로벌 현장학습(대학 단위 모집)
 - 전문대학 2학기 이상 수료한 재학생으로 누계 평점이 3.0/4.5 및 일정 기준 이상의 어학 능력을 갖추어 소속 대학을 통해 선정

○ WEST 프로그램(개별 모집)
 - 4년제 대학 4학기(전문대학 2학기) 이상 재·휴학생 및 프로그램 시작일 기준 1년 이내 졸업생 지원. 사업 선정 절차(어학 등 서류심사, 인성 및 영어면접)에 의해 선정
* WEST 프로그램은 기간에 따라 단기(6개월), 중기(12개월), 장기(18개월) 프로그램과  졸업생 대상 WEST 플러스(12개월)가 있으며 프로그램별 어학성적

In [ ]:
recommendation2(['여성', 27, '경기도', '파주시', '대학(원)졸업', '근로자/직장인', '', '미혼', '없음', None, '중위소득 40% 이하, 중위소득 40 ~ 60% 사이','근로자 지원, 개인금융지원, 문화생활 지원, 주택-부동산 지원, 의료 지원'])

===================user 정보=================
성별: 여성
나이: 27
시도: 경기도
시군구: 파주시
학력: 대학(원)졸업
직장: 근로자/직장인
가구원: 
결혼: 미혼
자녀: 없음
자녀상세: None
중위소득: 중위소득 40% 이하, 중위소득 40 ~ 60% 사이
관심정책: 근로자 지원, 개인금융지원, 문화생활 지원, 주택-부동산 지원, 의료 지원

1개가 13.0값으로 최적의 정책임
=====================policy 정보=================
84034
◎정책ID: 84035
◎정책서비스ID: B55265300020
◎서비스명: 산모 신생아 건강관리 모의계산
◎소관기관: 한국사회보장정보원
◎소관기관유형.1: 공공기관 등
◎생애주기: 청년||중장년
◎정책신청절차: None
◎선정기준: ○ 산모 및 배우자의 건강보험료 본인부담금 합산액이 전국 가구 기준 중위소득 120% 이하이며, 출산(예정)일 전 40일 또는 출산 후 30일 이내에 서비스를 신청한 산모를 지원합니다. 
 - 소득 기준을 초과하더라도 특수가정에 대해서는 지방자치단체별 예외지원* 가능(지방자치단체별 지원 여부에 대한 자세한 사항은 관할 보건소 문의)
* 희귀난치성 질환·장애인·새터민·결혼이민·미혼모, 분만취약지 산모, 쌍생아·셋째아 이상 출산 가정, 기타 소득 기준 완화 등
◎정책지원유형: 정보제공
◎서비스목적: 산모신생아 건강관리 대상 선정 여부를 예측하기 위한 모의계산 서비스 제공
◎지원내용: 산모신생아 건강관리 모의계산을 통해 서비스 수혜 가능성 확인



결혼·육아·교육>임신/출생>임산부지원
◎지원대상: ○ 산모 및 배우자의 건강보험료 본인부담금 합산액이 전국 가구 기준 중위소득 120% 이하이며, 출산(예정)일 전 40일 또는 출산 후 30일 이내에 있는 산모

여성
◎대상연령시작: 19
◎대상연령끝: 70
◎시도: 전국
◎시군구: None
◎중위소득: 0,10,20,30,40,50,60,70,80,90,100,110,120
◎관심